# 带掩码的

In [2]:
import csv
import numpy as np
from gurobipy import *
from CITY_GRAPH import *
from CITY_NODE import *
from ORDER import *
from VEHICLE import *
from tool_func import *
from Lower_Layer import *
import importlib
import tool_func
from update import *
import os
import time as tm
import copy
from my_env import *
import torch
import multiagent as magent
import matplotlib.pyplot as plt

"""这里是非强化学习部分"""
# 初始化
num_vehicle = 20
num_order = 6
num_city = 8
TIME = 144  # 
CAPACITY = 7
row = [10, 1, 3, 10]
Vehicles = {}
speed = 20 # 之前是20
cancel_penalty = 300
battery_consume = 10
battery_add = 300

matrix = np.tile(row, (num_vehicle, 1))

# 初始化
Vehicles = vehicle_generator(num_vehicle, num_city)
orders_unmatched = {}
G = CityGraph(num_city, 0.3, (10, 30))
name = "navie"
cancel_penalty = 300
order_canceled = 0
Total_order = {}

# 设置s_0
for time in range(TIME):
    Orders = order_generator(num_order, time, num_city-1, CAPACITY, G ,speed)
    for order in Orders.values():
        Total_order[order.id] = order

# 深复制最初的订单与车辆
prim_order = copy.deepcopy(Total_order)
prim_vehicle = copy.deepcopy(Vehicles)
"""这里是强化学习部分"""
# 超参数
STATE_DIM_VEHICLE = 11   # 车辆状态的特征维度
STATE_DIM_ORDER = 12     # 订单状态的特征维度
HIDDEN_DIM = 128         # 隐藏层维度
ACTION_DIM = num_city          # 动作空间维度
ACTOR_LR = 1e-2          # Actor 学习率
CRITIC_LR = 1e-2         # Critic 学习率
GAMMA = 0.99             # 折扣因子
NUM_EPISODES = 300     # 总训练轮数
# 这里也改了
STATE_DIM = 2 *HIDDEN_DIM       
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
agent = magent.MultiAgentAC(
    device = DEVICE,
    VEHICLE_STATE_DIM = STATE_DIM_VEHICLE,
    ORDER_STATE_DIM = STATE_DIM_ORDER, 
    NUM_CITIES = num_city, 
    HIDDEN_DIM = HIDDEN_DIM, 
    STATE_DIM = STATE_DIM
)
print(type(agent))
grid_rewards = []
# 开始计时
start_time = tm.time()

episode_reward = 0
ACTIONS = []
base_revenue = []
first_revnue = []
base_vehicle = []
base_vehicle_class = []
base_order_class = []
base_city_node = []
train_rewards = []
burn_in = 100
batch_time = int(TIME)
for episode in range(NUM_EPISODES):
    print(f"第{episode}次训练")
    Total_order = copy.deepcopy(prim_order)
    Vehicles = copy.deepcopy(prim_vehicle)
    objval = 0
    total_objval = 0
    reward = 0
    episode_reward = 0
    invalid_time =  0
    orders_unmatched = {} # 忘记加这个了
    orders_virtual = {}
    
    if episode > 0:
        # agent.load_model(load_path)
        if episode > 1:
            env.time = 0
        
        if episode < burn_in:
            greedy_epsilon = 0.6
            explore = True
        else:
            greedy_epsilon = 0.001
            explore = False
            if episode == burn_in:
                best_model = train_rewards.index(max(train_rewards))
                load_path = f"rl_para/model_checkpoint_{best_model+1}.pth"
                agent = torch.load(load_path,map_location="cpu")
                print(f"最优模型为{best_model}")
                agent.eval()
            if episode > burn_in:
                if train_rewards[best_model] > train_rewards[-1] or first_invalid < invalid_time:
                    agent = torch.load(f'rl_para/model_checkpoint_{best_model+1}.pth',map_location="cpu")
                    print(f"最优模型为{best_model}")
                    agent.eval()
                else:
                    best_model = train_rewards.index(train_rewards[-1]) 
                    agent = torch.load(f'rl_para/model_checkpoint_{best_model+1}.pth',map_location="cpu")
                    print(f"最优模型为{best_model}")
                    agent.eval()
    if_end = False
    for time in range(batch_time):
    
        group = [[], []]
        if time == batch_time -1:
            if_end = True
        # 按时间给出订单
        for order in Total_order.values():
            if order.start_time == time:
                orders_unmatched[order.id] = order
            # 加上这个代码后会导致性能降低
            """
            if order.matched is False:
                order.virtual_departure = order.departure 
            """
        if time != 0 and episode != 0:
            next_vehicle_states = vectorization_vehicle(Vehicles)
            # 改了，不再是total_order
            next_order_states = vectorization_order(orders_unmatched)
            # 这里防止梯度爆炸缩小了reward
            agent.update(vehicle_states, order_states, action,
                         grid_reward, next_vehicle_states, next_order_states , if_end)
            env.time = time
        if time == 0 :
            orders_virtual = orders_unmatched
           
            city_node = city_node_generator(G, orders_virtual, Vehicles, orders_unmatched)
            if episode == 1 :
                env = DispatchEnv(
                    G=G,
                    vehicles=Vehicles,
                    orders=Total_order,
                    cities=city_node,
                    capacity=CAPACITY
                )
            elif episode > 1:
                env.cities = city_node
           
            
        else:
            if episode == 0:
                city_update_without_drl(city_node , Vehicles, orders_unmatched ,time)
            else:
                city_update_without_drl(env.cities , Vehicles, orders_unmatched, time)
            
        if episode != 0:
            
            vehicle_states = vectorization_vehicle(Vehicles)
            # 这里也改了
            order_states = vectorization_order(orders_unmatched)
            
            greedy = random.randint(0, 1)
            if greedy > greedy_epsilon:
                greedy = True
            mask = env.get_mask(orders_unmatched)
            action = agent.take_action_mask(vehicle_states, order_states, mask, explore, greedy)
            reward = env.test_step(orders_unmatched,action)
         
            """
            COUNT = 1000
            max_reward = -999999
            max_action = action
            
           
            while reward != 1000 and COUNT > 0:
                greedy = random.randint(0, 1)
                if greedy > greedy_epsilon:
                    greedy = True
                action = agent.take_action(vehicle_states, order_states, explore, greedy)
                reward = env.test_step(orders_unmatched,action)
                COUNT -= 1
                if reward > max_reward:
                    max_reward = reward
                    max_action = action
            if COUNT == 0:
                reward = env.test_step(orders_unmatched, max_action)
            """
            
           
            
            ACTIONS.append(action) 
            """
            while reward != 0 :
                action = agent.take_action(vehicle_states, order_states)
                reward = env.test_step(orders_unmatched,action)
            
            """
            

        for vehicle in Vehicles.values():
            if vehicle.whether_city:
                group[0].append(vehicle.id)
            else:
                group[1].append(vehicle.id)

        if len(group[0]) != 0:

            if episode == 0:
                temp_Lower_Layer = Lower_Layer(G, city_node, Vehicles, orders_unmatched, name, group, time)
            else:
                temp_Lower_Layer = Lower_Layer(G, env.cities, Vehicles, orders_unmatched, name, group, time)
            try:
                temp_Lower_Layer.get_decision()
                temp_Lower_Layer.constrain_1()
                temp_Lower_Layer.constrain_2()
                temp_Lower_Layer.constrain_3()
                temp_Lower_Layer.constrain_4()
                temp_Lower_Layer.constrain_5()
                temp_Lower_Layer.model.setParam('OutputFlag', 0)
                total_penalty = cancel_penalty * order_canceled
                temp_Lower_Layer.set_objective(matrix)
            
                temp_Lower_Layer.model.optimize()

                if temp_Lower_Layer.model.status == GRB.OPTIMAL:
                    # save_results(temp_Lower_Layer,time)
                    # print("Objective value:", temp_Lower_Layer.model.objVal)
                    objval = temp_Lower_Layer.model.objVal 
                else:
                    temp_Lower_Layer.model.computeIIS()
                    temp_Lower_Layer.model.write('iis.ilp')  # 保存不可行约束
                    # print(f"{time}次，No optimal solution found.")
                    self_update(Vehicles, G)
                    objval = basic_cost(Vehicles, orders_unmatched)
                    
                
                _, var_order = temp_Lower_Layer.get_decision()
                update_var(temp_Lower_Layer, Vehicles, orders_unmatched)
                vehicle_in_city = update_vehicle(Vehicles, battery_consume, battery_add, speed, G)
                order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
            except:
                self_update(Vehicles, G)
               
                order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
                objval = - basic_cost(Vehicles, orders_unmatched)
                invalid_time += 1
            
        else:
            
            self_update(Vehicles, G)
            # print(f"{episode}轮，{time}次，{len(group[1])}辆车不在城市")
            order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
            objval = - basic_cost(Vehicles, orders_unmatched)
            # 利润（如果有）减去新增的取消订单
            
            invalid_time += 1
        objval = objval - update_order(orders_unmatched, time, speed) * cancel_penalty
        if episode != 0: 
            
            # 防止梯度爆炸
            grid_reward =   objval
            grid_rewards.append(reward)
            # print(grid_reward)
            episode_reward += reward + objval
        total_objval += objval

        if episode == 0:
            base_revenue.append(objval)
            base_vehicle.append(copy.deepcopy(group[0]))
            # base_vehicle_class.append(copy.deepcopy(Vehicles))
            # base_order_class.append(copy.deepcopy(Total_order))
            
            base_city_node.append(copy.deepcopy(city_node))
            """
            if episode == 1:
                first_revnue.append(objval)
            """
        else:
            # if base_revenue[time] != objval:
            #    print("base_revenue",time, objval,base_revenue[time])
            # if first_revnue[time] != objval:
            #    print("first_revenue",time, first_revnue[time], objval)
            """ 
            if base_vehicle[time] != group[0]:
                print("vehicle is different", len(base_vehicle[time]), len(group[0]))
            if base_city_node[time] != env.cities:
                print(time, base_city_node[time],"\n", env.cities)
            """
            
        # print(f"{len(orders_unmatched)}订单未被匹配,{order_canceled}订单超时,总利润为{objval},强化学习利润为{reward}")
    end_time = tm.time()
    execution_time = end_time - start_time
    if episode != 0:
        print(f"执行时间: {execution_time} 秒,{invalid_time}次未求解，当前强化学习值为{episode_reward},利润为{total_objval}")
        # torch.save(agent.state_dict(), 'model_checkpoint.pth')
        save_path = f"rl_para/model_checkpoint_{episode}.pth"
        torch.save(agent, save_path)
        train_rewards.append(total_objval)
        first_invalid = invalid_time
    else:
        print(f"未加强化学习利润为{total_objval},{invalid_time}次未求解")
    # grid_rewards.append(0)
    # save_path = f"actor_critic_model{episode}.pth"
    # load_path = f"actor_critic_model{episode}.pth"
    
plt.plot(grid_rewards, label='Grid Reward Curve')
plt.xlabel('Iteration')
plt.ylabel('Grid Reward')
plt.title('Reward Curve')
plt.legend()
plt.grid(True)
plt.show()
# print(find_duplicates_with_positions(ACTIONS))


<class 'multiagent.MultiAgentAC'>
第0次训练
未加强化学习利润为229195.0,109次未求解
第1次训练


AttributeError: 'MultiAgentAC' object has no attribute 'take_action_mask'

# 调试版 1

In [ ]:
import csv
import numpy as np
from gurobipy import *
from CITY_GRAPH import *
from CITY_NODE import *
from ORDER import *
from VEHICLE import *
from tool_func import *
from Lower_Layer import *
import SETTING
import importlib
import tool_func
from update import *
import os
import time as tm
import copy
from my_env import *
import torch
import multiagent as magent
import matplotlib.pyplot as plt

"""这里是非强化学习部分"""
# 初始化
num_vehicle = 20
num_order = 6
num_city = 8
TIME = 144  # 
CAPACITY = 7
row = [10, 1, 3, 10]
Vehicles = {}
speed = 20 # 之前是20
cancel_penalty = 300
battery_consume = 10
battery_add = 300

matrix = np.tile(row, (num_vehicle, 1))

# 初始化
Vehicles = vehicle_generator(num_vehicle, num_city)
orders_unmatched = {}
G = CityGraph(num_city, 0.3, (10, 30))
name = "navie"
cancel_penalty = 300
order_canceled = 0
Total_order = {}

# 设置s_0
for time in range(TIME):
    Orders = order_generator(num_order, time, num_city-1, CAPACITY, G ,speed)
    for order in Orders.values():
        Total_order[order.id] = order

# 深复制最初的订单与车辆
prim_order = copy.deepcopy(Total_order)
prim_vehicle = copy.deepcopy(Vehicles)
"""这里是强化学习部分"""
# 超参数
STATE_DIM_VEHICLE = 11   # 车辆状态的特征维度
STATE_DIM_ORDER = 12     # 订单状态的特征维度
HIDDEN_DIM = 128         # 隐藏层维度
ACTION_DIM = num_city          # 动作空间维度
ACTOR_LR = 1e-2          # Actor 学习率
CRITIC_LR = 1e-2         # Critic 学习率
GAMMA = 0.99             # 折扣因子
NUM_EPISODES = 300     # 总训练轮数
# 这里也改了
STATE_DIM = 2 *HIDDEN_DIM       
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
agent = magent.MultiAgentAC(
    device = DEVICE,
    VEHICLE_STATE_DIM = STATE_DIM_VEHICLE,
    ORDER_STATE_DIM = STATE_DIM_ORDER, 
    NUM_CITIES = num_city, 
    HIDDEN_DIM = HIDDEN_DIM, 
    STATE_DIM = STATE_DIM
)
print(type(agent))
grid_rewards = []
# 开始计时
start_time = tm.time()

episode_reward = 0
ACTIONS = []
base_revenue = []
first_revnue = []
base_vehicle = []
base_vehicle_class = []
base_order_class = []
base_city_node = []
train_rewards = []
burn_in = 30
batch_time = int(TIME/2)
for episode in range(NUM_EPISODES):
    print(f"第{episode}次训练")
    Total_order = copy.deepcopy(prim_order)
    Vehicles = copy.deepcopy(prim_vehicle)
    objval = 0
    total_objval = 0
    reward = 0
    episode_reward = 0
    invalid_time =  0
    orders_unmatched = {} # 忘记加这个了
    orders_virtual = {}
    
    if episode > 0:
        # agent.load_model(load_path)
        if episode > 1:
            env.time = 0
        
        if episode < burn_in:
            greedy_epsilon = 0.6
            explore = True
        else:
            greedy_epsilon = 0
            explore = False
            
            if episode == burn_in:
                best_model = train_rewards.index(max(train_rewards))
                load_path = f"rl_para/model_checkpoint_{best_model+1}.pth"
                agent = torch.load(load_path,map_location="cpu")
                print(f"最优模型为{best_model}")
                agent.eval()
            """
            if episode > burn_in:
                if train_rewards[best_model] > train_rewards[-1] or first_invalid < invalid_time:
                    agent = torch.load(f'rl_para/model_checkpoint_{best_model+1}.pth',map_location="cpu")
                    print(f"最优模型为{best_model}")
                    agent.eval()
                else:
                    best_model = train_rewards.index(train_rewards[-1]) 
                    agent = torch.load(f'rl_para/model_checkpoint_{best_model+1}.pth',map_location="cpu")
                    print(f"最优模型为{best_model}")
                    agent.eval()
            """
    if_end = False
    for time in range(batch_time):
    
        group = [[], []]
        if time == batch_time -1:
            if_end = True
        # 按时间给出订单
        for order in Total_order.values():
            if order.start_time == time:
                orders_unmatched[order.id] = order
            # 加上这个代码后会导致性能降低
            """
            if order.matched is False:
                order.virtual_departure = order.departure 
            """
        if time != 0 and episode != 0:
            next_vehicle_states = vectorization_vehicle(Vehicles)
            # 改了，不再是total_order
            next_order_states = vectorization_order(orders_unmatched)
            # 这里防止梯度爆炸缩小了reward
            agent.update(vehicle_states, order_states, action,
                         grid_reward, next_vehicle_states, next_order_states , if_end)
            env.time = time
        if time == 0 :
            orders_virtual = orders_unmatched
           
            city_node = city_node_generator(G, orders_virtual, Vehicles, orders_unmatched)
            if episode == 1 :
                env = DispatchEnv(
                    G=G,
                    vehicles=Vehicles,
                    orders=Total_order,
                    cities=city_node,
                    capacity=CAPACITY
                )
            elif episode > 1:
                env.cities = city_node
           
            
        else:
            if episode == 0:
                city_update_without_drl(city_node , Vehicles, orders_unmatched ,time)
            else:
                city_update_without_drl(env.cities , Vehicles, orders_unmatched, time)
            
        if episode != 0:
            
            vehicle_states = vectorization_vehicle(Vehicles)
            # 这里也改了
            order_states = vectorization_order(orders_unmatched)
            
            greedy = random.randint(0, 1)
            if greedy > greedy_epsilon:
                greedy = True
            mask = env.get_mask(orders_unmatched)
            action , logits = agent.take_action_mask(vehicle_states, order_states, mask, explore, greedy)
            # reward = env.test_step(orders_unmatched,action)
         
            """
            COUNT = 1000
            max_reward = -999999
            max_action = action
            
           
            while reward != 1000 and COUNT > 0:
                greedy = random.randint(0, 1)
                if greedy > greedy_epsilon:
                    greedy = True
                action = agent.take_action(vehicle_states, order_states, explore, greedy)
                reward = env.test_step(orders_unmatched,action)
                COUNT -= 1
                if reward > max_reward:
                    max_reward = reward
                    max_action = action
            if COUNT == 0:
                reward = env.test_step(orders_unmatched, max_action)
            """
            
           
            
            ACTIONS.append(action) 
            """
            while reward != 0 :
                action = agent.take_action(vehicle_states, order_states)
                reward = env.test_step(orders_unmatched,action)
            
            """
            

        for vehicle in Vehicles.values():
            if vehicle.whether_city:
                group[0].append(vehicle.id)
            else:
                group[1].append(vehicle.id)

        if len(group[0]) != 0:

            if episode == 0:
                temp_Lower_Layer = Lower_Layer(G, city_node, Vehicles, orders_unmatched, name, group, time)
            else:
                temp_Lower_Layer = Lower_Layer(G, env.cities, Vehicles, orders_unmatched, name, group, time)
            try:
                temp_Lower_Layer.get_decision()
                temp_Lower_Layer.constrain_1()
                temp_Lower_Layer.constrain_2()
                temp_Lower_Layer.constrain_3()
                temp_Lower_Layer.constrain_4()
                temp_Lower_Layer.constrain_5()
                temp_Lower_Layer.model.setParam('OutputFlag', 0)
                total_penalty = cancel_penalty * order_canceled
                temp_Lower_Layer.set_objective(matrix)
            
                temp_Lower_Layer.model.optimize()

                if temp_Lower_Layer.model.status == GRB.OPTIMAL:
                    # save_results(temp_Lower_Layer,time)
                    # print("Objective value:", temp_Lower_Layer.model.objVal)
                    objval = temp_Lower_Layer.model.objVal 
                else:
                    temp_Lower_Layer.model.computeIIS()
                    temp_Lower_Layer.model.write('iis.ilp')  # 保存不可行约束
                    # print(f"{time}次，No optimal solution found.")
                    self_update(Vehicles, G)
                    objval = basic_cost(Vehicles, orders_unmatched)
                    
                
                _, var_order = temp_Lower_Layer.get_decision()
                update_var(temp_Lower_Layer, Vehicles, orders_unmatched)
                vehicle_in_city = update_vehicle(Vehicles, battery_consume, battery_add, speed, G)
                order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
            except:
                self_update(Vehicles, G)
               
                order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
                objval = - basic_cost(Vehicles, orders_unmatched)
                invalid_time += 1
            
        else:
            
            self_update(Vehicles, G)
            # print(f"{episode}轮，{time}次，{len(group[1])}辆车不在城市")
            order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
            objval = - basic_cost(Vehicles, orders_unmatched)
            # 利润（如果有）减去新增的取消订单
            
            invalid_time += 1
        objval = objval - update_order(orders_unmatched, time, speed) * cancel_penalty
        if episode != 0: 
            
            # 防止梯度爆炸
            grid_reward =   objval
            # grid_rewards.append(reward)
            # print(grid_reward)
            episode_reward +=  objval
        total_objval += objval

        if episode == 0:
            base_revenue.append(objval)
            base_vehicle.append(copy.deepcopy(group[0]))
            # base_vehicle_class.append(copy.deepcopy(Vehicles))
            # base_order_class.append(copy.deepcopy(Total_order))
            
            base_city_node.append(copy.deepcopy(city_node))
            """
            if episode == 1:
                first_revnue.append(objval)
            """
        else:
            # if base_revenue[time] != objval:
            #    print("base_revenue",time, objval,base_revenue[time])
            # if first_revnue[time] != objval:
            #    print("first_revenue",time, first_revnue[time], objval)
            """ 
            if base_vehicle[time] != group[0]:
                print("vehicle is different", len(base_vehicle[time]), len(group[0]))
            if base_city_node[time] != env.cities:
                print(time, base_city_node[time],"\n", env.cities)
            """
            
        # print(f"{len(orders_unmatched)}订单未被匹配,{order_canceled}订单超时,总利润为{objval},强化学习利润为{reward}")
    end_time = tm.time()
    execution_time = end_time - start_time
    if episode != 0:
        print(f"执行时间: {execution_time} 秒,{invalid_time}次未求解，当前强化学习值为{episode_reward},利润为{total_objval}")
        # torch.save(agent.state_dict(), 'model_checkpoint.pth')
        # save_path = f"rl_para/model_checkpoint_{episode}.pth"
        # torch.save(agent, save_path)
        train_rewards.append(total_objval)
        first_invalid = invalid_time
    else:
        print(f"未加强化学习利润为{total_objval},{invalid_time}次未求解")
    # grid_rewards.append(0)
    # save_path = f"actor_critic_model{episode}.pth"
    # load_path = f"actor_critic_model{episode}.pth"
    
plt.plot(grid_rewards, label='Grid Reward Curve')
plt.xlabel('Iteration')
plt.ylabel('Grid Reward')
plt.title('Reward Curve')
plt.legend()
plt.grid(True)
plt.show()
# print(find_duplicates_with_positions(ACTIONS))

<class 'multiagent.MultiAgentAC'>
第0次训练
Set parameter LicenseID to value 2584673


INFO:gurobipy:Set parameter LicenseID to value 2584673


未加强化学习利润为333835.0,48次未求解
第1次训练
执行时间: 1.7765674591064453 秒,48次未求解，当前强化学习值为333835.0,利润为333835.0
第2次训练
执行时间: 2.8372347354888916 秒,48次未求解，当前强化学习值为333835.0,利润为333835.0
第3次训练
执行时间: 3.9000909328460693 秒,48次未求解，当前强化学习值为333835.0,利润为333835.0
第4次训练
执行时间: 4.9130539894104 秒,48次未求解，当前强化学习值为333835.0,利润为333835.0
第5次训练
执行时间: 5.973233938217163 秒,48次未求解，当前强化学习值为333835.0,利润为333835.0
第6次训练
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000],
        [0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5000, 0.0000, 0.5000],
        [0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.0000, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.7870, 0.0000, 0.1065, 0.0000, 0.1065, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3333, 0.3333, 0.3333],
        [0.0000, 0.0000, 0.0000, 0.0

UnboundLocalError: local variable 'actions' referenced before assignment

# 调试版2

In [ ]:
import csv
import numpy as np
from gurobipy import *
from CITY_GRAPH import *
from CITY_NODE import *
from ORDER import *
from VEHICLE import *
from tool_func import *
from Lower_Layer import *
import SETTING
import importlib
import tool_func
from update import *
import os
import time as tm
import copy
from my_env import *
import torch
import multiagent as magent
import matplotlib.pyplot as plt

"""这里是非强化学习部分"""
# 初始化
num_vehicle = 20
num_order = 6
num_city = 8
TIME = 144  # 
CAPACITY = 7
row = [10, 1, 3, 10]
Vehicles = {}
speed = 20 # 之前是20
cancel_penalty = 300
battery_consume = 10
battery_add = 300

matrix = np.tile(row, (num_vehicle, 1))

# 初始化
Vehicles = vehicle_generator(num_vehicle, num_city)
orders_unmatched = {}
G = CityGraph(num_city, 0.3, (10, 30))
name = "navie"
cancel_penalty = 300
order_canceled = 0
Total_order = {}

# 设置s_0
for time in range(TIME):
    Orders = order_generator(num_order, time, num_city-1, CAPACITY, G ,speed)
    for order in Orders.values():
        Total_order[order.id] = order

# 深复制最初的订单与车辆
prim_order = copy.deepcopy(Total_order)
prim_vehicle = copy.deepcopy(Vehicles)
"""这里是强化学习部分"""
# 超参数
STATE_DIM_VEHICLE = 11   # 车辆状态的特征维度
STATE_DIM_ORDER = 12     # 订单状态的特征维度
HIDDEN_DIM = 128         # 隐藏层维度
ACTION_DIM = num_city          # 动作空间维度
ACTOR_LR = 1e-2          # Actor 学习率
CRITIC_LR = 1e-2         # Critic 学习率
GAMMA = 0.99             # 折扣因子
NUM_EPISODES = 300     # 总训练轮数
# 这里也改了
STATE_DIM = 2 *HIDDEN_DIM       
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
agent = magent.MultiAgentAC(
    device = DEVICE,
    VEHICLE_STATE_DIM = STATE_DIM_VEHICLE,
    ORDER_STATE_DIM = STATE_DIM_ORDER, 
    NUM_CITIES = num_city, 
    HIDDEN_DIM = HIDDEN_DIM, 
    STATE_DIM = STATE_DIM
)
print(type(agent))
grid_rewards = []
# 开始计时
start_time = tm.time()

episode_reward = 0
ACTIONS = []
base_revenue = []
first_revnue = []
base_vehicle = []
base_vehicle_class = []
base_order_class = []
base_city_node = []
train_rewards = []
burn_in = 100
batch_time = int(TIME/2)
for episode in range(NUM_EPISODES):
    print(f"第{episode}次训练")
    Total_order = copy.deepcopy(prim_order)
    Vehicles = copy.deepcopy(prim_vehicle)
    objval = 0
    total_objval = 0
    reward = 0
    episode_reward = 0
    invalid_time =  0
    orders_unmatched = {} # 忘记加这个了
    orders_virtual = {}
    
    if episode > 0:
        # agent.load_model(load_path)
        if episode > 1:
            env.time = 0
        
        if episode < burn_in:
            greedy_epsilon = 0.6
            explore = True
        else:
            greedy_epsilon = 0.001
            explore = False
            if episode == burn_in:
                best_model = train_rewards.index(max(train_rewards))
                load_path = f"rl_para/model_checkpoint_{best_model+1}.pth"
                agent = torch.load(load_path,map_location="cpu")
                print(f"最优模型为{best_model}")
                agent.eval()
            if episode > burn_in:
                if train_rewards[best_model] > train_rewards[-1] or first_invalid < invalid_time:
                    agent = torch.load(f'rl_para/model_checkpoint_{best_model+1}.pth',map_location="cpu")
                    print(f"最优模型为{best_model}")
                    agent.eval()
                else:
                    best_model = train_rewards.index(train_rewards[-1]) 
                    agent = torch.load(f'rl_para/model_checkpoint_{best_model+1}.pth',map_location="cpu")
                    print(f"最优模型为{best_model}")
                    agent.eval()
    if_end = False
    for time in range(batch_time):
    
        group = [[], []]
        if time == batch_time -1:
            if_end = True
        # 按时间给出订单
        for order in Total_order.values():
            if order.start_time == time:
                orders_unmatched[order.id] = order
            # 加上这个代码后会导致性能降低
            """
            if order.matched is False:
                order.virtual_departure = order.departure 
            """
        if time != 0 and episode != 0:
            next_vehicle_states = vectorization_vehicle(Vehicles)
            # 改了，不再是total_order
            next_order_states = vectorization_order(orders_unmatched)
            # 这里防止梯度爆炸缩小了reward
            agent.update_logtis(vehicle_states, order_states, logits,
                         grid_reward, next_vehicle_states, next_order_states , if_end)
            env.time = time
        if time == 0 :
            orders_virtual = orders_unmatched
           
            city_node = city_node_generator(G, orders_virtual, Vehicles, orders_unmatched)
            if episode == 1 :
                env = DispatchEnv(
                    G=G,
                    vehicles=Vehicles,
                    orders=Total_order,
                    cities=city_node,
                    capacity=CAPACITY
                )
            elif episode > 1:
                env.cities = city_node
           
            
        else:
            if episode == 0:
                city_update_without_drl(city_node , Vehicles, orders_unmatched ,time)
            else:
                city_update_without_drl(env.cities , Vehicles, orders_unmatched, time)
            
        if episode != 0:
            
            vehicle_states = vectorization_vehicle(Vehicles)
            # 这里也改了
            order_states = vectorization_order(orders_unmatched)
            
            greedy = random.randint(0, 1)
            if greedy > greedy_epsilon:
                greedy = True
            mask = env.get_mask(orders_unmatched)
            action, logits = agent.take_action_mask_special(vehicle_states, order_states, mask, explore, greedy)
            reward = env.test_step(orders_unmatched,action)
         
            """
            COUNT = 1000
            max_reward = -999999
            max_action = action
            
           
            while reward != 1000 and COUNT > 0:
                greedy = random.randint(0, 1)
                if greedy > greedy_epsilon:
                    greedy = True
                action = agent.take_action(vehicle_states, order_states, explore, greedy)
                reward = env.test_step(orders_unmatched,action)
                COUNT -= 1
                if reward > max_reward:
                    max_reward = reward
                    max_action = action
            if COUNT == 0:
                reward = env.test_step(orders_unmatched, max_action)
            """
            
           
            
            ACTIONS.append(action) 
            """
            while reward != 0 :
                action = agent.take_action(vehicle_states, order_states)
                reward = env.test_step(orders_unmatched,action)
            
            """
            

        for vehicle in Vehicles.values():
            if vehicle.whether_city:
                group[0].append(vehicle.id)
            else:
                group[1].append(vehicle.id)

        if len(group[0]) != 0:

            if episode == 0:
                temp_Lower_Layer = Lower_Layer(G, city_node, Vehicles, orders_unmatched, name, group, time)
            else:
                temp_Lower_Layer = Lower_Layer(G, env.cities, Vehicles, orders_unmatched, name, group, time)
            try:
                temp_Lower_Layer.get_decision()
                temp_Lower_Layer.constrain_1()
                temp_Lower_Layer.constrain_2()
                temp_Lower_Layer.constrain_3()
                temp_Lower_Layer.constrain_4()
                temp_Lower_Layer.constrain_5()
                temp_Lower_Layer.model.setParam('OutputFlag', 0)
                total_penalty = cancel_penalty * order_canceled
                temp_Lower_Layer.set_objective(matrix)
            
                temp_Lower_Layer.model.optimize()

                if temp_Lower_Layer.model.status == GRB.OPTIMAL:
                    # save_results(temp_Lower_Layer,time)
                    # print("Objective value:", temp_Lower_Layer.model.objVal)
                    objval = temp_Lower_Layer.model.objVal 
                else:
                    temp_Lower_Layer.model.computeIIS()
                    temp_Lower_Layer.model.write('iis.ilp')  # 保存不可行约束
                    # print(f"{time}次，No optimal solution found.")
                    self_update(Vehicles, G)
                    objval = basic_cost(Vehicles, orders_unmatched)
                    
                
                _, var_order = temp_Lower_Layer.get_decision()
                update_var(temp_Lower_Layer, Vehicles, orders_unmatched)
                vehicle_in_city = update_vehicle(Vehicles, battery_consume, battery_add, speed, G)
                order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
            except:
                self_update(Vehicles, G)
               
                order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
                objval = - basic_cost(Vehicles, orders_unmatched)
                invalid_time += 1
            
        else:
            
            self_update(Vehicles, G)
            # print(f"{episode}轮，{time}次，{len(group[1])}辆车不在城市")
            order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
            objval = - basic_cost(Vehicles, orders_unmatched)
            # 利润（如果有）减去新增的取消订单
            
            invalid_time += 1
        objval = objval - update_order(orders_unmatched, time, speed) * cancel_penalty
        if episode != 0: 
            
            # 防止梯度爆炸
            grid_reward =   objval
            grid_rewards.append(reward)
            # print(grid_reward)
            episode_reward += reward + objval
        total_objval += objval

        if episode == 0:
            base_revenue.append(objval)
            base_vehicle.append(copy.deepcopy(group[0]))
            # base_vehicle_class.append(copy.deepcopy(Vehicles))
            # base_order_class.append(copy.deepcopy(Total_order))
            
            base_city_node.append(copy.deepcopy(city_node))
            """
            if episode == 1:
                first_revnue.append(objval)
            """
        else:
            # if base_revenue[time] != objval:
            #    print("base_revenue",time, objval,base_revenue[time])
            # if first_revnue[time] != objval:
            #    print("first_revenue",time, first_revnue[time], objval)
            """ 
            if base_vehicle[time] != group[0]:
                print("vehicle is different", len(base_vehicle[time]), len(group[0]))
            if base_city_node[time] != env.cities:
                print(time, base_city_node[time],"\n", env.cities)
            """
            
        # print(f"{len(orders_unmatched)}订单未被匹配,{order_canceled}订单超时,总利润为{objval},强化学习利润为{reward}")
    end_time = tm.time()
    execution_time = end_time - start_time
    if episode != 0:
        print(f"执行时间: {execution_time} 秒,{invalid_time}次未求解，当前强化学习值为{episode_reward},利润为{total_objval}")
        # torch.save(agent.state_dict(), 'model_checkpoint.pth')
        save_path = f"rl_para/model_checkpoint_{episode}.pth"
        torch.save(agent, save_path)
        train_rewards.append(total_objval)
        first_invalid = invalid_time
    else:
        print(f"未加强化学习利润为{total_objval},{invalid_time}次未求解")
    # grid_rewards.append(0)
    # save_path = f"actor_critic_model{episode}.pth"
    # load_path = f"actor_critic_model{episode}.pth"
    
plt.plot(grid_rewards, label='Grid Reward Curve')
plt.xlabel('Iteration')
plt.ylabel('Grid Reward')
plt.title('Reward Curve')
plt.legend()
plt.grid(True)
plt.show()
# print(find_duplicates_with_positions(ACTIONS))

<class 'multiagent.MultiAgentAC'>
第0次训练
未加强化学习利润为200830.0,53次未求解
第1次训练
v_encoded 出现 NaN，输入状态可能异常！
o_encoded 出现 NaN，输入状态可能异常！
actor_input 出现 NaN，输入状态可能异常！
global vehicle 出现 NaN，输入状态可能异常！
repeated global 出现 NaN，输入状态可能异常！
tensor([[nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan]], grad_fn=<SoftmaxBackward0>)
logits 出现 NaN，输入状态可能异常！


UnboundLocalError: local variable 'actions' referenced before assignment

## 示例1 burn in后探索

In [ ]:
import plotly.express as px
import pandas as pd
px.line(pd.DataFrame({'Iteration': range(len(train_rewards)), 'Reward': train_rewards}), 
        x='Iteration', y='Reward', title='Total Reward Curve').show()

## 示例2 burn in后不探索

In [ ]:
import plotly.express as px
import pandas as pd
px.line(pd.DataFrame({'Iteration': range(len(train_rewards)), 'Reward': train_rewards}), 
        x='Iteration', y='Reward', title='Total Reward Curve').show()

INFO:numexpr.utils:NumExpr defaulting to 16 threads.


## 示例3 更新条件

In [ ]:
import plotly.express as px
import pandas as pd
px.line(pd.DataFrame({'Iteration': range(len(train_rewards)), 'Reward': train_rewards}), 
        x='Iteration', y='Reward', title='Total Reward Curve').show()

# 带跳过的

In [ ]:
import csv
import numpy as np
from gurobipy import *
from CITY_GRAPH import *
from CITY_NODE import *
from ORDER import *
from VEHICLE import *
from tool_func import *
from Lower_Layer import *
import SETTING
import importlib
import tool_func
from update import *
import os
import time as tm
import copy
from my_env import *
import torch
import multiagent as magent
import matplotlib.pyplot as plt

"""这里是非强化学习部分"""
# 初始化
num_vehicle = 20
num_order = 6
num_city = 8
TIME = 144  # 
CAPACITY = 7
row = [10, 1, 3, 10]
Vehicles = {}
speed = 20 # 之前是20
cancel_penalty = 300
battery_consume = 10
battery_add = 300

matrix = np.tile(row, (num_vehicle, 1))

# 初始化
Vehicles = vehicle_generator(num_vehicle, num_city)
orders_unmatched = {}
G = CityGraph(num_city, 0.3, (10, 30))
name = "navie"
cancel_penalty = 300
order_canceled = 0
Total_order = {}

# 设置s_0
for time in range(TIME):
    Orders = order_generator(num_order, time, num_city-1, CAPACITY, G ,speed)
    for order in Orders.values():
        Total_order[order.id] = order

# 深复制最初的订单与车辆
prim_order = copy.deepcopy(Total_order)
prim_vehicle = copy.deepcopy(Vehicles)
"""这里是强化学习部分"""
# 超参数
STATE_DIM_VEHICLE = 11   # 车辆状态的特征维度
STATE_DIM_ORDER = 12     # 订单状态的特征维度
HIDDEN_DIM = 128         # 隐藏层维度
ACTION_DIM = num_city          # 动作空间维度
ACTOR_LR = 1e-2          # Actor 学习率
CRITIC_LR = 1e-2         # Critic 学习率
GAMMA = 0.99             # 折扣因子
NUM_EPISODES = 300     # 总训练轮数
# 这里也改了
STATE_DIM = 2 *HIDDEN_DIM       
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
agent = magent.MultiAgentAC(
    device = DEVICE,
    VEHICLE_STATE_DIM = STATE_DIM_VEHICLE,
    ORDER_STATE_DIM = STATE_DIM_ORDER, 
    NUM_CITIES = num_city, 
    HIDDEN_DIM = HIDDEN_DIM, 
    STATE_DIM = STATE_DIM
)
print(type(agent))
grid_rewards = []
# 开始计时
start_time = tm.time()

episode_reward = 0
ACTIONS = []
base_revenue = []
first_revnue = []
base_vehicle = []
base_vehicle_class = []
base_order_class = []
base_city_node = []
train_rewards = []
burn_in = 100
batch_time = int(TIME/2)
for episode in range(NUM_EPISODES):
    print(f"第{episode}次训练")
    Total_order = copy.deepcopy(prim_order)
    Vehicles = copy.deepcopy(prim_vehicle)
    objval = 0
    total_objval = 0
    reward = 0
    episode_reward = 0
    invalid_time =  0
    orders_unmatched = {} # 忘记加这个了
    orders_virtual = {}
    
    if episode > 0:
        # agent.load_model(load_path)
        if episode > 1:
            env.time = 0
        
        if episode < burn_in:
            greedy_epsilon = 0.6
            explore = True
        else:
            greedy_epsilon = 0.001
            explore = False
            if episode == burn_in:
                best_model = train_rewards.index(max(train_rewards))
                agent = torch.load(f'model_checkpoint_{best_model+1}.pth',map_location="cpu")
                print(f"最优模型为{best_model}")
                agent.eval()
            if episode > burn_in:
                if train_rewards[best_model] > train_rewards[-1] or first_invalid >= invalid_time:
                    agent = torch.load(f'model_checkpoint_{best_model+1}.pth',map_location="cpu")
                    print(f"最优模型为{best_model}")
                    agent.eval()
                else:
                    best_model = train_rewards.index(train_rewards[-1]) 
                    agent = torch.load(f'model_checkpoint_{best_model+1}.pth',map_location="cpu")
                    print(f"最优模型为{best_model}")
                    agent.eval()
    if_end = False
    for time in range(batch_time):
    
        group = [[], []]
        if time == batch_time -1:
            if_end = True
        # 按时间给出订单
        for order in Total_order.values():
            if order.start_time == time:
                orders_unmatched[order.id] = order
            # 加上这个代码后会导致性能降低
            """
            if order.matched is False:
                order.virtual_departure = order.departure 
            """
        if time != 0 and episode != 0:
            next_vehicle_states = vectorization_vehicle(Vehicles)
            # 改了，不再是total_order
            next_order_states = vectorization_order(orders_unmatched)
            # 这里防止梯度爆炸缩小了reward
            agent.update(vehicle_states, order_states, action,
                         grid_reward, next_vehicle_states, next_order_states , if_end)
            env.time = time
        if time == 0 :
            orders_virtual = orders_unmatched
           
            city_node = city_node_generator(G, orders_virtual, Vehicles, orders_unmatched)
            if episode == 1 :
                env = DispatchEnv(
                    G=G,
                    vehicles=Vehicles,
                    orders=Total_order,
                    cities=city_node,
                    capacity=CAPACITY
                )
            elif episode > 1:
                env.cities = city_node
           
            
        else:
            if episode == 0:
                city_update_without_drl(city_node , Vehicles, orders_unmatched ,time)
            else:
                city_update_without_drl(env.cities , Vehicles, orders_unmatched, time)
            
        if episode != 0:
            
            vehicle_states = vectorization_vehicle(Vehicles)
            # 这里也改了
            order_states = vectorization_order(orders_unmatched)
            
            greedy = random.randint(0, 1)
            if greedy > greedy_epsilon:
                greedy = True
            action = agent.take_action(vehicle_states, order_states, explore, greedy)
            reward = env.test_step(orders_unmatched,action)
            # 一个循环代码让我达到最优
            # 屁股后面的代码是为了让我达到最优
            # 这里是为了让我达到最优
            # 现在放弃了重采样
            """
            COUNT = 1000
            max_reward = -999999
            max_action = action
            
           
            while reward != 1000 and COUNT > 0:
                greedy = random.randint(0, 1)
                if greedy > greedy_epsilon:
                    greedy = True
                action = agent.take_action(vehicle_states, order_states, explore, greedy)
                reward = env.test_step(orders_unmatched,action)
                COUNT -= 1
                if reward > max_reward:
                    max_reward = reward
                    max_action = action
            if COUNT == 0:
                reward = env.test_step(orders_unmatched, max_action)
            """
            
           
            
            ACTIONS.append(action) 
            """
            while reward != 0 :
                action = agent.take_action(vehicle_states, order_states)
                reward = env.test_step(orders_unmatched,action)
            
            """
            

        for vehicle in Vehicles.values():
            if vehicle.whether_city:
                group[0].append(vehicle.id)
            else:
                group[1].append(vehicle.id)

        if len(group[0]) != 0:

            if episode == 0:
                temp_Lower_Layer = Lower_Layer(G, city_node, Vehicles, orders_unmatched, name, group, time)
            else:
                temp_Lower_Layer = Lower_Layer(G, env.cities, Vehicles, orders_unmatched, name, group, time)
            try:
                temp_Lower_Layer.get_decision()
                temp_Lower_Layer.constrain_1()
                temp_Lower_Layer.constrain_2()
                temp_Lower_Layer.constrain_3()
                temp_Lower_Layer.constrain_4()
                temp_Lower_Layer.constrain_5()
                temp_Lower_Layer.model.setParam('OutputFlag', 0)
                total_penalty = cancel_penalty * order_canceled
                temp_Lower_Layer.set_objective(matrix)
            
                temp_Lower_Layer.model.optimize()

                if temp_Lower_Layer.model.status == GRB.OPTIMAL:
                    # save_results(temp_Lower_Layer,time)
                    # print("Objective value:", temp_Lower_Layer.model.objVal)
                    objval = temp_Lower_Layer.model.objVal 
                else:
                    temp_Lower_Layer.model.computeIIS()
                    temp_Lower_Layer.model.write('iis.ilp')  # 保存不可行约束
                    # print(f"{time}次，No optimal solution found.")
                    self_update(Vehicles, G)
                    objval = basic_cost(Vehicles, orders_unmatched)
                    
                
                _, var_order = temp_Lower_Layer.get_decision()
                update_var(temp_Lower_Layer, Vehicles, orders_unmatched)
                vehicle_in_city = update_vehicle(Vehicles, battery_consume, battery_add, speed, G)
                order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
            except:
                self_update(Vehicles, G)
               
                order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
                objval = - basic_cost(Vehicles, orders_unmatched)
                invalid_time += 1
            
        else:
            
            self_update(Vehicles, G)
            # print(f"{episode}轮，{time}次，{len(group[1])}辆车不在城市")
            order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
            objval = - basic_cost(Vehicles, orders_unmatched)
            # 利润（如果有）减去新增的取消订单
            
            invalid_time += 1
        objval = objval - update_order(orders_unmatched, time, speed) * cancel_penalty
        if episode != 0: 
            
            # 防止梯度爆炸
            grid_reward =   objval
            grid_rewards.append(reward)
            # print(grid_reward)
            episode_reward += reward + objval
        total_objval += objval

        if episode == 0:
            base_revenue.append(objval)
            base_vehicle.append(copy.deepcopy(group[0]))
            # base_vehicle_class.append(copy.deepcopy(Vehicles))
            # base_order_class.append(copy.deepcopy(Total_order))
            
            base_city_node.append(copy.deepcopy(city_node))
            """
            if episode == 1:
                first_revnue.append(objval)
            """
        else:
            # if base_revenue[time] != objval:
            #    print("base_revenue",time, objval,base_revenue[time])
            # if first_revnue[time] != objval:
            #    print("first_revenue",time, first_revnue[time], objval)
            """ 
            if base_vehicle[time] != group[0]:
                print("vehicle is different", len(base_vehicle[time]), len(group[0]))
            if base_city_node[time] != env.cities:
                print(time, base_city_node[time],"\n", env.cities)
            """
            
        # print(f"{len(orders_unmatched)}订单未被匹配,{order_canceled}订单超时,总利润为{objval},强化学习利润为{reward}")
    end_time = tm.time()
    execution_time = end_time - start_time
    if episode != 0:
        print(f"执行时间: {execution_time} 秒,{invalid_time}次未求解，当前强化学习值为{episode_reward},利润为{total_objval}")
        # torch.save(agent.state_dict(), 'model_checkpoint.pth')
        torch.save(agent, f"model_checkpoint_{episode}.pth")
        train_rewards.append(total_objval)
        first_invalid = invalid_time
    else:
        print(f"未加强化学习利润为{total_objval},{invalid_time}次未求解")
    # grid_rewards.append(0)
    # save_path = f"actor_critic_model{episode}.pth"
    # load_path = f"actor_critic_model{episode}.pth"
    
plt.plot(grid_rewards, label='Grid Reward Curve')
plt.xlabel('Iteration')
plt.ylabel('Grid Reward')
plt.title('Reward Curve')
plt.legend()
plt.grid(True)
plt.show()
# print(find_duplicates_with_positions(ACTIONS))

<class 'multiagent.MultiAgentAC'>
第0次训练
Set parameter LicenseID to value 2584673


INFO:gurobipy:Set parameter LicenseID to value 2584673


未加强化学习利润为351240.0,48次未求解
第1次训练
执行时间: 1.5951976776123047 秒,66次未求解，当前强化学习值为-316088.0,利润为-1388.0
第2次训练
执行时间: 2.8220858573913574 秒,40次未求解，当前强化学习值为255049.0,利润为471649.0
第3次训练
执行时间: 3.92783522605896 秒,48次未求解，当前强化学习值为76507.0,利润为333107.0
第4次训练
执行时间: 4.801097393035889 秒,60次未求解，当前强化学习值为-187197.0,利润为108603.0
第5次训练
执行时间: 6.016491651535034 秒,45次未求解，当前强化学习值为168199.0,利润为405999.0
第6次训练
执行时间: 6.894177436828613 秒,60次未求解，当前强化学习值为-183913.0,利润为112087.0
第7次训练
执行时间: 7.8495588302612305 秒,60次未求解，当前强化学习值为-186198.0,利润为108602.0
第8次训练
执行时间: 9.370670318603516 秒,43次未求解，当前强化学习值为195294.0,利润为423494.0
第9次训练
执行时间: 10.391007423400879 秒,59次未求解，当前强化学习值为-181762.0,利润为112638.0
第10次训练
执行时间: 11.553189754486084 秒,57次未求解，当前强化学习值为-102270.0,利润为178130.0
第11次训练
执行时间: 13.124795198440552 秒,47次未求解，当前强化学习值为114564.0,利润为361264.0
第12次训练
执行时间: 14.306801080703735 秒,60次未求解，当前强化学习值为-180413.0,利润为112087.0
第13次训练
执行时间: 15.57046389579773 秒,48次未求解，当前强化学习值为68873.0,利润为326673.0
第14次训练
执行时间: 16.64451813697815 秒,57次未求解，当前强化学习值为-110057.0,利润为170643.0
第15次训练


KeyboardInterrupt: 

# ALNS

In [ ]:
import numpy as np
from alns import ALNS, State
from alns.accept import SimulatedAnnealing
from alns.stop import MaxIterations
from alns.select import RouletteWheel
import random

# Problem Data
np.random.seed(42)
num_customers = 10
num_vehicles = 3
capacity = 100

# 随机生成坐标和需求
depot = np.array([50, 50])
nodes = np.random.randint(0, 100, (num_customers, 2))
demands = np.random.randint(5, 20, num_customers)

# 计算欧几里得距离
def distance(a, b):
    return np.linalg.norm(a - b)

distance_matrix = np.zeros((num_customers + 1, num_customers + 1))
nodes_full = np.vstack([depot, nodes])  # 加入仓库
for i in range(len(nodes_full)):
    for j in range(len(nodes_full)):
        distance_matrix[i, j] = distance(nodes_full[i], nodes_full[j])

# 初始状态
class VRPState(State):
    def __init__(self, routes):
        self.routes = routes
    
    def objective(self):
        total_cost = sum(
            distance_matrix[route[i], route[i+1]]
            for route in self.routes for i in range(len(route)-1)
        )
        return total_cost
    
    def copy(self):
        return VRPState([route[:] for route in self.routes])

# 破坏算子
def random_removal(state, rng, num_remove=2):
    if isinstance(num_remove, np.random.Generator):  
        num_remove = rng.integers(1, 4)  

    new_state = state.copy()
    for _ in range(num_remove):
        if any(new_state.routes):
            route = random.choice(new_state.routes)
            if len(route) > 2:  # 只有在长度 > 2 时才移除
                idx = rng.integers(1, max(2, len(route) - 1))  # 确保 idx 合法
                route.pop(idx)
    return new_state



# 修复算子
def greedy_insert(state, rng):
    new_state = state.copy()
    unassigned = [i for i in range(1, num_customers + 1) if not any(i in r for r in new_state.routes)]
    for i in unassigned:
        best_cost = float('inf')
        best_route = None
        best_position = None
        for route in new_state.routes:
            for pos in range(1, len(route)):
                temp_route = route[:pos] + [i] + route[pos:]
                cost = sum(distance_matrix[temp_route[j], temp_route[j+1]] for j in range(len(temp_route)-1))
                if cost < best_cost:
                    best_cost, best_route, best_position = cost, route, pos
        if best_route is not None:
            best_route.insert(best_position, i)
    return new_state

# ALNS 运行
initial_routes = [[0, i, 0] for i in range(1, num_customers + 1)]  # 每个客户单独一辆车
initial_state = VRPState(initial_routes)
alns = ALNS()
alns.add_destroy_operator(random_removal)
alns.add_repair_operator(greedy_insert)

# 设定接受准则（模拟退火）
accept = SimulatedAnnealing(1000, 1, 500, method="linear")
select = RouletteWheel([1] * 4, 0.8, 1, 1)
stop = MaxIterations(1000)

result = alns.iterate(initial_state, select, accept, stop)

# 输出最优解
best_state = result.best_state
print("Best routes:", best_state.routes)
print("Best cost:", best_state.objective())

INFO:alns.ALNS:Finished iterating in 0.15s.


Best routes: [[0, 1, 0], [0, 2, 0], [0, 3, 0], [0, 4, 0], [0, 5, 0], [0, 6, 0], [0, 7, 0], [0, 8, 0], [0, 9, 0], [0, 10, 0]]
Best cost: 858.1198159028768


# 数据固定的逐步调试

In [ ]:
import csv
import numpy as np
from gurobipy import *
from CITY_GRAPH import *
from CITY_NODE import *
from ORDER import *
from VEHICLE import *
from tool_func import *
from Lower_Layer import *
import importlib
import tool_func
from update import *
import os
import time as tm
import copy
from my_env import *
import torch
import multiagent as magent
import matplotlib.pyplot as plt
import pickle
# 假设有如下初始化函数

"""这里是非强化学习部分"""
# 初始化
num_vehicle = 20
num_order = 3
num_city = 8
TIME = 144  # 
CAPACITY = 7
row = [10, 1, 3, 10]
Vehicles = {}
speed = 20 # 之前是20
cancel_penalty = 300
battery_consume = 10
battery_add = 300

matrix = np.tile(row, (num_vehicle, 1))
Vehicles = vehicle_generator(num_vehicle, num_city)
orders_unmatched = {}
G = CityGraph(num_city, 0.3, (10, 30))
name = "navie"
cancel_penalty = 300
order_canceled = 0
Total_order = {}

# 生成固定样本
for time in range(TIME):
    Orders = order_generator(num_order, time, num_city-1, CAPACITY, G, speed)
    for order in Orders.values():
        Total_order[order.id] = order

# 保存样本到本地
with open('sample_data.pkl', 'wb') as f:
    pickle.dump({
        'Vehicles': Vehicles,
        'Total_order': Total_order,
        'G':G
    }, f)

# 基本实验

In [ ]:
import csv
import numpy as np
from gurobipy import *
from CITY_GRAPH import *
from CITY_NODE import *
from ORDER import *
from VEHICLE import *
from tool_func import *
from Lower_Layer import *
import importlib

from update import *
import os
import time as tm
import copy
from my_env import *
import torch
import MARL_BASE as magent
import matplotlib.pyplot as plt
import pickle

with open('sample_data.pkl', 'rb') as f:
    data = pickle.load(f)
    Vehicles = data['Vehicles']
    Total_order = data['Total_order']
    G = data['G']
# 初始化
num_vehicle = 20
num_order = 6
num_city = 8
TIME = 144  # 
CAPACITY = 7
row = [10, 1, 3, 10]
speed = 20 # 之前是20
cancel_penalty = 300
battery_consume = 10
battery_add = 300

matrix = np.tile(row, (num_vehicle, 1))
orders_unmatched = {}
name = "navie"
cancel_penalty = 300
order_canceled = 0


# 深复制最初的订单与车辆
prim_order = copy.deepcopy(Total_order)
prim_vehicle = copy.deepcopy(Vehicles)
"""这里是强化学习部分"""
# 超参数
STATE_DIM_VEHICLE = 11   # 车辆状态的特征维度
STATE_DIM_ORDER = 12     # 订单状态的特征维度
HIDDEN_DIM = 128         # 隐藏层维度
ACTION_DIM = num_city          # 动作空间维度
ACTOR_LR = 1e-2          # Actor 学习率
CRITIC_LR = 1e-2         # Critic 学习率
GAMMA = 0.99             # 折扣因子
NUM_EPISODES = 300     # 总训练轮数
# 这里也改了
STATE_DIM = 2 *HIDDEN_DIM       
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
agent = magent.MultiAgentAC(
    device = DEVICE,
    VEHICLE_STATE_DIM = STATE_DIM_VEHICLE,
    ORDER_STATE_DIM = STATE_DIM_ORDER, 
    NUM_CITIES = num_city, 
    HIDDEN_DIM = HIDDEN_DIM, 
    STATE_DIM = STATE_DIM_VEHICLE+STATE_DIM_ORDER
)
print(type(agent))
grid_rewards = []
# 开始计时
start_time = tm.time()

episode_reward = 0
ACTIONS = []
base_revenue = []
first_revnue = []
base_vehicle = []
base_vehicle_class = []
base_order_class = []
base_city_node = []
train_rewards = []
burn_in = 100
batch_time = int(TIME)
greedy_epsilon = 0.7
for episode in range(NUM_EPISODES):
    print(f"第{episode}次训练")
    Total_order = copy.deepcopy(prim_order)
    Vehicles = copy.deepcopy(prim_vehicle)
    objval = 0
    total_objval = 0
    reward = 0
    episode_reward = 0
    invalid_time =  0
    orders_unmatched = {} # 忘记加这个了
    orders_virtual = {}
    
    if episode > 0:
        if episode > 1:
            env.time = 0
        
        if episode < burn_in:
            explore = True
        else:
            greedy_epsilon = greedy_epsilon * greedy_epsilon
            explore = False
    if_end = False
    for time in range(batch_time):
    
        group = [[], []]
        if time == batch_time -1:
            if_end = True
        # 按时间给出订单
        for order in Total_order.values():
            if order.start_time == time:
                orders_unmatched[order.id] = order
            # 加上这个代码后会导致性能降低
        for vehicle in Vehicles.values():
            if vehicle.whether_city:
                group[0].append(vehicle.id)
            else:
                group[1].append(vehicle.id)
        
        if time != 0 and episode != 0:
            
            next_vehicle_states = vectorization_vehicle(Vehicles)
            # 改了，不再是total_order
            next_order_states = vectorization_order(orders_unmatched)
            # 这里防止梯度爆炸缩小了reward
            if last_vehicle:
                
                agent.update_third(vehicle_states, order_states, action, selected_log_probs, log_probs, probs,
                            grid_reward, next_vehicle_states, next_order_states , if_end)
            env.time = time
        if time == 0 :
            orders_virtual = orders_unmatched
           
            city_node = city_node_generator(G, orders_virtual, Vehicles, orders_unmatched)
            if episode == 1 :
                env = DispatchEnv(
                    G=G,
                    vehicles=Vehicles,
                    orders=Total_order,
                    cities=city_node,
                    capacity=CAPACITY
                )
            elif episode > 1:
                env.cities = city_node
           
            
        else:
            if episode == 0:
                city_update_without_drl(city_node , Vehicles, orders_unmatched ,time)
            else:
                city_update_without_drl(env.cities , Vehicles, orders_unmatched, time)
            
        if episode != 0:
            
            vehicle_states = vectorization_vehicle(Vehicles)
            # 这里也改了
            order_states = vectorization_order(orders_unmatched)
            if len(group[0]) != 0:
                greedy = random.randint(0, 1)
                if greedy > greedy_epsilon:
                    greedy = True
                mask = env.get_mask(orders_unmatched)
                # 注意这里有多种take_action形式
                action, selected_log_probs, log_probs, probs = agent.take_action_vehicle(vehicle_states, order_states, mask, explore, greedy)
                reward = env.test_step(orders_unmatched,action)
            
                ACTIONS.append(action) 

        

        if len(group[0]) != 0:
            last_vehicle = True
            if episode == 0:
                temp_Lower_Layer = Lower_Layer(G, city_node, Vehicles, orders_unmatched, name, group, time)
            else:
                temp_Lower_Layer = Lower_Layer(G, env.cities, Vehicles, orders_unmatched, name, group, time)
            try:
                temp_Lower_Layer.get_decision()
                temp_Lower_Layer.constrain_1()
                temp_Lower_Layer.constrain_2()
                temp_Lower_Layer.constrain_3()
                temp_Lower_Layer.constrain_4()
                temp_Lower_Layer.constrain_5()
                temp_Lower_Layer.model.setParam('OutputFlag', 0)
                total_penalty = cancel_penalty * order_canceled
                temp_Lower_Layer.set_objective(matrix)
            
                temp_Lower_Layer.model.optimize()

                if temp_Lower_Layer.model.status == GRB.OPTIMAL:
                    # save_results(temp_Lower_Layer,time)
                    # print("Objective value:", temp_Lower_Layer.model.objVal)
                    objval = temp_Lower_Layer.model.objVal 
                else:
                    temp_Lower_Layer.model.computeIIS()
                    temp_Lower_Layer.model.write('iis.ilp')  # 保存不可行约束
                    # print(f"{time}次，No optimal solution found.")
                    self_update(Vehicles, G)
                    objval = basic_cost(Vehicles, orders_unmatched)
                    
                
                _, var_order = temp_Lower_Layer.get_decision()
                update_var(temp_Lower_Layer, Vehicles, orders_unmatched)
                vehicle_in_city = update_vehicle(Vehicles, battery_consume, battery_add, speed, G)
                order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
            except:
                self_update(Vehicles, G)
               
                order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
                objval = - basic_cost(Vehicles, orders_unmatched)
                invalid_time += 1
            
        else :
            last_vehicle = False
            self_update(Vehicles, G)
            order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
            objval = - basic_cost(Vehicles, orders_unmatched)
            invalid_time += 1
        objval = objval - update_order(orders_unmatched, time, speed) * cancel_penalty
        if episode != 0: 
            
            # grid_reward =   (objval-base_revenue[time])/1000
            grid_reward =   objval
            grid_rewards.append(reward)
            episode_reward += reward + objval
        total_objval += objval

        if episode == 0:
            base_revenue.append(objval)
            base_vehicle.append(copy.deepcopy(group[0]))
            base_city_node.append(copy.deepcopy(city_node))
           
       
    end_time = tm.time()
    execution_time = end_time - start_time
    if episode != 0:
        print(f"执行时间: {execution_time} 秒,{invalid_time}次未求解，当前强化学习值为{episode_reward},利润为{total_objval}")
        save_path = f"rl_para/model_checkpoint_{episode}.pth"
        torch.save(agent, save_path)
        train_rewards.append(total_objval)
        first_invalid = invalid_time
    else:
        print(f"未加强化学习利润为{total_objval},{invalid_time}次未求解")

<class 'MARL_BASE.MultiAgentAC'>
第0次训练
Set parameter LicenseID to value 2584673


INFO:gurobipy:Set parameter LicenseID to value 2584673


未加强化学习利润为569776.0,101次未求解
第1次训练
[28, 21, 21, 7, 3, 14, 28, 1]
[28, 10, 21, 7, 0, 14, 21, 0]
[22, 7, 15, 7, 0, 14, 14, 0]
[21, 8, 14, 7, 0, 7, 7, 0]
[21, 7, 0, 7, 0, 7, 7, 0]
[21, 7, 0, 2, 7, 7, 7, 0]
[15, 7, 0, 0, 7, 1, 0, 0]
[7, 7, 0, 0, 7, 0, 0, 0]
[1, 0, 0, 0, 4, 7, 0, 0]
[0, 0, 0, 0, 0, 3, 0, 0]
[0, 0, 0, 1, 1, 0, 0, 0]
[0, 0, 3, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 2, 0]
[0, 0, 3, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 3, 0]
[0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 2, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
import plotly.express as px
import pandas as pd
px.line(pd.DataFrame({'Iteration': range(len(train_rewards)), 'Reward': train_rewards}), 
        x='Iteration', y='Reward', title='Total Reward Curve').show()

INFO:numexpr.utils:NumExpr defaulting to 16 threads.


# PPO

In [ ]:
import csv
import numpy as np
from gurobipy import *
from CITY_GRAPH import *
from CITY_NODE import *
from ORDER import *
from VEHICLE import *
from tool_func import *
from Lower_Layer import *
import importlib

from update import *
import os
import time as tm
import copy
from my_env import *
import torch
import ppo as magent
import matplotlib.pyplot as plt
import pickle

with open('sample_data.pkl', 'rb') as f:
    data = pickle.load(f)
    Vehicles = data['Vehicles']
    Total_order = data['Total_order']
    G = data['G']
# 初始化
num_vehicle = 20
num_order = 6
num_city = 8
TIME = 144  # 
CAPACITY = 7
row = [10, 1, 3, 10]
speed = 20 # 之前是20
cancel_penalty = 300
battery_consume = 10
battery_add = 300

matrix = np.tile(row, (num_vehicle, 1))
orders_unmatched = {}
name = "navie"
cancel_penalty = 300
order_canceled = 0


# 深复制最初的订单与车辆
prim_order = copy.deepcopy(Total_order)
prim_vehicle = copy.deepcopy(Vehicles)
"""这里是强化学习部分"""
# 超参数
STATE_DIM_VEHICLE = 11   # 车辆状态的特征维度
STATE_DIM_ORDER = 12     # 订单状态的特征维度
HIDDEN_DIM = 32         # 隐藏层维度
ACTION_DIM = num_city          # 动作空间维度
ACTOR_LR = 1e-2          # Actor 学习率
CRITIC_LR = 1e-2         # Critic 学习率
GAMMA = 0.99             # 折扣因子
NUM_EPISODES = 300     # 总训练轮数
# 这里也改了
STATE_DIM = 2 *HIDDEN_DIM       
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
agent = magent.MultiAgentAC(
    device = DEVICE,
    VEHICLE_STATE_DIM = STATE_DIM_VEHICLE,
    ORDER_STATE_DIM = STATE_DIM_ORDER, 
    NUM_CITIES = num_city, 
    HIDDEN_DIM = HIDDEN_DIM, 
    STATE_DIM = STATE_DIM_VEHICLE+STATE_DIM_ORDER
)
print(type(agent))
grid_rewards = []
# 开始计时
start_time = tm.time()

episode_reward = 0
ACTIONS = []
base_revenue = []
first_revnue = []
base_vehicle = []
base_vehicle_class = []
base_order_class = []
base_city_node = []
train_rewards = []
burn_in = 100
batch_time = int(TIME)
greedy_epsilon = 0.7
for episode in range(NUM_EPISODES):
    print(f"第{episode}次训练")
    Total_order = copy.deepcopy(prim_order)
    Vehicles = copy.deepcopy(prim_vehicle)
    objval = 0
    total_objval = 0
    reward = 0
    episode_reward = 0
    invalid_time =  0
    orders_unmatched = {} # 忘记加这个了
    orders_virtual = {}
    
    if episode > 0:
        if episode > 1:
            env.time = 0
        
        if episode < burn_in:
            explore = True
        else:
            greedy_epsilon = greedy_epsilon * greedy_epsilon
            explore = False
    if_end = False
    for time in range(batch_time):
    
        group = [[], []]
        if time == batch_time -1:
            if_end = True
        # 按时间给出订单
        for order in Total_order.values():
            if order.start_time == time:
                orders_unmatched[order.id] = order
            # 加上这个代码后会导致性能降低
        for vehicle in Vehicles.values():
            if vehicle.whether_city:
                group[0].append(vehicle.id)
            else:
                group[1].append(vehicle.id)
        
        if time != 0 and episode != 0:
            
            next_vehicle_states = vectorization_vehicle(Vehicles)
            # 改了，不再是total_order
            next_order_states = vectorization_order(orders_unmatched)
            # 这里防止梯度爆炸缩小了reward
            if last_vehicle:
                
                agent.update_third(vehicle_states, order_states, action, selected_log_probs, log_probs, probs,
                            grid_reward, next_vehicle_states, next_order_states , if_end)
            env.time = time
        if time == 0 :
            orders_virtual = orders_unmatched
           
            city_node = city_node_generator(G, orders_virtual, Vehicles, orders_unmatched)
            if episode == 1 :
                env = DispatchEnv(
                    G=G,
                    vehicles=Vehicles,
                    orders=Total_order,
                    cities=city_node,
                    capacity=CAPACITY
                )
            elif episode > 1:
                env.cities = city_node
           
            
        else:
            if episode == 0:
                city_update_without_drl(city_node , Vehicles, orders_unmatched ,time)
            else:
                city_update_without_drl(env.cities , Vehicles, orders_unmatched, time)
            
        if episode != 0:
            
            vehicle_states = vectorization_vehicle(Vehicles)
            # 这里也改了
            order_states = vectorization_order(orders_unmatched)
            if len(group[0]) != 0:
                greedy = random.randint(0, 1)
                if greedy > greedy_epsilon:
                    greedy = True
                mask = env.get_mask(orders_unmatched)
                # 注意这里有多种take_action形式
                action, selected_log_probs, log_probs, probs = agent.take_action_vehicle(vehicle_states, order_states, mask, explore, greedy)
                reward = env.test_step(orders_unmatched,action)
            
                ACTIONS.append(action) 

        

        if len(group[0]) != 0:
            last_vehicle = True
            if episode == 0:
                temp_Lower_Layer = Lower_Layer(G, city_node, Vehicles, orders_unmatched, name, group, time)
            else:
                temp_Lower_Layer = Lower_Layer(G, env.cities, Vehicles, orders_unmatched, name, group, time)
            try:
                temp_Lower_Layer.get_decision()
                temp_Lower_Layer.constrain_1()
                temp_Lower_Layer.constrain_2()
                temp_Lower_Layer.constrain_3()
                temp_Lower_Layer.constrain_4()
                temp_Lower_Layer.constrain_5()
                temp_Lower_Layer.model.setParam('OutputFlag', 0)
                total_penalty = cancel_penalty * order_canceled
                temp_Lower_Layer.set_objective(matrix)
            
                temp_Lower_Layer.model.optimize()

                if temp_Lower_Layer.model.status == GRB.OPTIMAL:
                    # save_results(temp_Lower_Layer,time)
                    # print("Objective value:", temp_Lower_Layer.model.objVal)
                    objval = temp_Lower_Layer.model.objVal 
                else:
                    temp_Lower_Layer.model.computeIIS()
                    temp_Lower_Layer.model.write('iis.ilp')  # 保存不可行约束
                    # print(f"{time}次，No optimal solution found.")
                    self_update(Vehicles, G)
                    objval = basic_cost(Vehicles, orders_unmatched)
                    
                
                _, var_order = temp_Lower_Layer.get_decision()
                update_var(temp_Lower_Layer, Vehicles, orders_unmatched)
                vehicle_in_city = update_vehicle(Vehicles, battery_consume, battery_add, speed, G)
                order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
            except:
                self_update(Vehicles, G)
               
                order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
                objval = - basic_cost(Vehicles, orders_unmatched)
                invalid_time += 1
            
        else :
            last_vehicle = False
            self_update(Vehicles, G)
            order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
            objval = - basic_cost(Vehicles, orders_unmatched)
            invalid_time += 1
        objval = objval - update_order(orders_unmatched, time, speed) * cancel_penalty
        if episode != 0: 
            
            grid_reward =   (objval-base_revenue[time])/1000
            grid_rewards.append(reward)
            episode_reward += reward + objval
        total_objval += objval

        if episode == 0:
            base_revenue.append(objval)
            base_vehicle.append(copy.deepcopy(group[0]))
            base_city_node.append(copy.deepcopy(city_node))
           
       
    end_time = tm.time()
    execution_time = end_time - start_time
    if episode != 0:
        print(f"执行时间: {execution_time} 秒,{invalid_time}次未求解，当前强化学习值为{episode_reward},利润为{total_objval}")
        save_path = f"rl_para/model_checkpoint_{episode}.pth"
        torch.save(agent, save_path)
        train_rewards.append(total_objval)
        first_invalid = invalid_time
    else:
        print(f"未加强化学习利润为{total_objval},{invalid_time}次未求解")

<class 'ppo.MultiAgentAC'>
第0次训练
Set parameter LicenseID to value 2584673


INFO:gurobipy:Set parameter LicenseID to value 2584673


未加强化学习利润为569776.0,101次未求解
第1次训练
执行时间: 2.099555492401123 秒,125次未求解，当前强化学习值为109770.0,利润为109770.0
第2次训练
执行时间: 3.600386381149292 秒,84次未求解，当前强化学习值为909874.0,利润为909874.0
第3次训练
执行时间: 4.82237696647644 秒,95次未求解，当前强化学习值为697123.0,利润为697123.0
第4次训练
执行时间: 5.321692228317261 秒,128次未求解，当前强化学习值为50047.0,利润为50047.0
第5次训练
执行时间: 6.3974316120147705 秒,101次未求解，当前强化学习值为581304.0,利润为581304.0
第6次训练
执行时间: 6.854481935501099 秒,128次未求解，当前强化学习值为48468.0,利润为48468.0
第7次训练
执行时间: 7.520401954650879 秒,118次未求解，当前强化学习值为258541.0,利润为258541.0
第8次训练
执行时间: 8.176277875900269 秒,118次未求解，当前强化学习值为255749.0,利润为255749.0
第9次训练
执行时间: 8.810486793518066 秒,122次未求解，当前强化学习值为162252.0,利润为162252.0
第10次训练
执行时间: 9.574955701828003 秒,113次未求解，当前强化学习值为340865.0,利润为340865.0
第11次训练
执行时间: 13.014147758483887 秒,21次未求解，当前强化学习值为2125014.0,利润为2125014.0
第12次训练
执行时间: 13.391227960586548 秒,135次未求解，当前强化学习值为-96516.0,利润为-96516.0
第13次训练
执行时间: 13.979040145874023 秒,124次未求解，当前强化学习值为128873.0,利润为128873.0
第14次训练
执行时间: 15.122689008712769 秒,98次未求解，当前强化学习值为640858.0,利润为640858.0
第15次训

# buffer版

In [ ]:
import csv
import numpy as np
from gurobipy import *
from CITY_GRAPH import *
from CITY_NODE import *
from ORDER import *
from VEHICLE import *
from tool_func import *
from Lower_Layer import *
import importlib

from update import *
import os
import time as tm
import copy
from my_env import *
import torch
import aloha_buffer as magent
import matplotlib.pyplot as plt
import pickle

with open('sample_data.pkl', 'rb') as f:
    data = pickle.load(f)
    Vehicles = data['Vehicles']
    Total_order = data['Total_order']
    G = data['G']
# 初始化
num_vehicle = 20
num_order = 6
num_city = 8
TIME = 144  # 
CAPACITY = 7
row = [10, 1, 3, 10]
speed = 20 # 之前是20
cancel_penalty = 300
battery_consume = 10
battery_add = 300

matrix = np.tile(row, (num_vehicle, 1))
orders_unmatched = {}
name = "navie"
cancel_penalty = 300
order_canceled = 0


# 深复制最初的订单与车辆
prim_order = copy.deepcopy(Total_order)
prim_vehicle = copy.deepcopy(Vehicles)
"""这里是强化学习部分"""
# 超参数
STATE_DIM_VEHICLE = 11   # 车辆状态的特征维度
STATE_DIM_ORDER = 12     # 订单状态的特征维度
HIDDEN_DIM = 64         # 隐藏层维度
ACTION_DIM = num_city          # 动作空间维度
ACTOR_LR = 1e-2          # Actor 学习率
CRITIC_LR = 1e-2         # Critic 学习率
GAMMA = 0.99             # 折扣因子
NUM_EPISODES = 300     # 总训练轮数
# 这里也改了
STATE_DIM = 2 *HIDDEN_DIM       
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
agent = magent.MultiAgentAC(
    device = DEVICE,
    VEHICLE_STATE_DIM = STATE_DIM_VEHICLE,
    ORDER_STATE_DIM =  num_city, 
    NUM_CITIES = num_city, 
    HIDDEN_DIM = HIDDEN_DIM, 
    STATE_DIM = num_city *2
)
print(type(agent))
grid_rewards = []
# 开始计时
start_time = tm.time()

episode_reward = 0
ACTIONS = []
base_revenue = []
first_revnue = []
base_vehicle = []
base_vehicle_class = []
base_order_class = []
base_city_node = []
train_rewards = []
burn_in = 100
batch_time = int(TIME)
greedy_epsilon = 0.7
for episode in range(NUM_EPISODES):
    print(f"第{episode}次训练")
    Total_order = copy.deepcopy(prim_order)
    Vehicles = copy.deepcopy(prim_vehicle)
    objval = 0
    total_objval = 0
    reward = 0
    episode_reward = 0
    invalid_time =  0
    orders_unmatched = {} # 忘记加这个了
    orders_virtual = {}
    
    if episode > 0:
        if episode > 1:
            env.time = 0
        
        if episode < burn_in:
            explore = True
        else:
            greedy_epsilon = greedy_epsilon * greedy_epsilon
            explore = False
    if_end = False
    for time in range(batch_time):
    
        group = [[], []]
        if time == batch_time -1:
            if_end = True
        # 按时间给出订单
        for order in Total_order.values():
            if order.start_time == time:
                orders_unmatched[order.id] = order
            # 加上这个代码后会导致性能降低
        for vehicle in Vehicles.values():
            if vehicle.whether_city:
                group[0].append(vehicle.id)
            else:
                group[1].append(vehicle.id)
        
        if time != 0 and episode != 0:
            
            next_vehicle_states = seat_count(CAPACITY, city_node)
            # 改了，不再是total_order
            next_order_states = vectorization_order_mask(orders_unmatched, G, num_city)
            # 这里防止梯度爆炸缩小了reward
            if last_vehicle:
                grid_reward = np.clip(grid_reward, 0, 2)
                agent.store_experience(vehicle_states, order_states, selected_log_probs, 
                       log_probs, probs, grid_reward, next_vehicle_states, next_order_states)
                agent.update_third_buffer_rnn()
            env.time = time
        if time == 0 :
            orders_virtual = orders_unmatched
           
            city_node = city_node_generator(G, orders_virtual, Vehicles, orders_unmatched)
            if episode == 1 :
                env = DispatchEnv(
                    G=G,
                    vehicles=Vehicles,
                    orders=Total_order,
                    cities=city_node,
                    capacity=CAPACITY
                )
            elif episode > 1:
                env.cities = city_node
           
            
        else:
            if episode == 0:
                city_update_without_drl(city_node , Vehicles, orders_unmatched ,time)
            else:
                city_update_without_drl(env.cities , Vehicles, orders_unmatched, time)
            
        if episode != 0:
            
            vehicle_states = seat_count(CAPACITY, city_node)
            # 这里也改了
            order_states = vectorization_order_mask(orders_unmatched, G, num_city)
            if len(group[0]) != 0:
                greedy = random.randint(0, 1)
                if greedy > greedy_epsilon:
                    greedy = True
                mask = env.get_mask(orders_unmatched)
                action, selected_log_probs, log_probs, probs = agent.take_action_third(vehicle_states, order_states, mask, explore, greedy)
                reward = env.test_step(orders_unmatched,action)
            
                ACTIONS.append(action) 

        

        if len(group[0]) != 0:
            last_vehicle = True
            if episode == 0:
                temp_Lower_Layer = Lower_Layer(G, city_node, Vehicles, orders_unmatched, name, group, time)
            else:
                temp_Lower_Layer = Lower_Layer(G, env.cities, Vehicles, orders_unmatched, name, group, time)
            try:
                temp_Lower_Layer.get_decision()
                temp_Lower_Layer.constrain_1()
                temp_Lower_Layer.constrain_2()
                temp_Lower_Layer.constrain_3()
                temp_Lower_Layer.constrain_4()
                temp_Lower_Layer.constrain_5()
                temp_Lower_Layer.model.setParam('OutputFlag', 0)
                total_penalty = cancel_penalty * order_canceled
                temp_Lower_Layer.set_objective(matrix)
            
                temp_Lower_Layer.model.optimize()

                if temp_Lower_Layer.model.status == GRB.OPTIMAL:
                    # save_results(temp_Lower_Layer,time)
                    # print("Objective value:", temp_Lower_Layer.model.objVal)
                    objval = temp_Lower_Layer.model.objVal 
                else:
                    temp_Lower_Layer.model.computeIIS()
                    temp_Lower_Layer.model.write('iis.ilp')  # 保存不可行约束
                    # print(f"{time}次，No optimal solution found.")
                    self_update(Vehicles, G)
                    objval = basic_cost(Vehicles, orders_unmatched)
                    
                
                _, var_order = temp_Lower_Layer.get_decision()
                update_var(temp_Lower_Layer, Vehicles, orders_unmatched)
                vehicle_in_city = update_vehicle(Vehicles, battery_consume, battery_add, speed, G)
                order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
            except:
                self_update(Vehicles, G)
               
                order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
                objval = - basic_cost(Vehicles, orders_unmatched)
                invalid_time += 1
            
        else :
            last_vehicle = False
            self_update(Vehicles, G)
            order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
            objval = - basic_cost(Vehicles, orders_unmatched)
            invalid_time += 1
        objval = objval - update_order(orders_unmatched, time, speed) * cancel_penalty
        if episode != 0: 
            
            grid_reward =   objval-base_revenue[time]
            grid_rewards.append(reward)
            episode_reward += reward + objval
        total_objval += objval

        if episode == 0:
            base_revenue.append(objval)
            base_vehicle.append(copy.deepcopy(group[0]))
            base_city_node.append(copy.deepcopy(city_node))
           
       
    end_time = tm.time()
    execution_time = end_time - start_time
    if episode != 0:
        print(f"执行时间: {execution_time} 秒,{invalid_time}次未求解，当前强化学习值为{episode_reward},利润为{total_objval}")
        save_path = f"rl_para/model_checkpoint_{episode}.pth"
        torch.save(agent, save_path)
        train_rewards.append(total_objval)
        first_invalid = invalid_time
    else:
        print(f"未加强化学习利润为{total_objval},{invalid_time}次未求解")

<class 'aloha_buffer.MultiAgentAC'>
第0次训练
Set parameter LicenseID to value 2584673


INFO:gurobipy:Set parameter LicenseID to value 2584673


未加强化学习利润为569776.0,101次未求解
第1次训练
执行时间: 1.8205502033233643 秒,135次未求解，当前强化学习值为-96525.0,利润为-96525.0
第2次训练


TypeError: expected Tensor as element 0 in argument 0, but got PackedSequence

# 订单输入是掩码

In [ ]:
import csv
import numpy as np
from gurobipy import *
from CITY_GRAPH import *
from CITY_NODE import *
from ORDER import *
from VEHICLE import *
from tool_func import *
from Lower_Layer import *
import importlib

from update import *
import os
import time as tm
import copy
from my_env import *
import torch
import multiagent as magent
import matplotlib.pyplot as plt
import pickle

with open('sample_data.pkl', 'rb') as f:
    data = pickle.load(f)
    Vehicles = data['Vehicles']
    Total_order = data['Total_order']
    G = data['G']
# 初始化
num_vehicle = 20
num_order = 6
num_city = 8
TIME = 144  # 
CAPACITY = 7
row = [10, 1, 3, 10]
speed = 20 # 之前是20
cancel_penalty = 300
battery_consume = 10
battery_add = 300

matrix = np.tile(row, (num_vehicle, 1))
orders_unmatched = {}
name = "navie"
cancel_penalty = 300
order_canceled = 0


# 深复制最初的订单与车辆
prim_order = copy.deepcopy(Total_order)
prim_vehicle = copy.deepcopy(Vehicles)
"""这里是强化学习部分"""
# 超参数
STATE_DIM_VEHICLE = 11   # 车辆状态的特征维度
STATE_DIM_ORDER = 12     # 订单状态的特征维度
HIDDEN_DIM = 64         # 隐藏层维度
ACTION_DIM = num_city          # 动作空间维度
ACTOR_LR = 1e-2          # Actor 学习率
CRITIC_LR = 1e-2         # Critic 学习率
GAMMA = 0.99             # 折扣因子
NUM_EPISODES = 300     # 总训练轮数
# 这里也改了
STATE_DIM = 2 *HIDDEN_DIM       
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
agent = magent.MultiAgentAC(
    device = DEVICE,
    VEHICLE_STATE_DIM = STATE_DIM_VEHICLE,
    ORDER_STATE_DIM =  num_city, 
    NUM_CITIES = num_city, 
    HIDDEN_DIM = HIDDEN_DIM, 
    STATE_DIM = STATE_DIM
)
print(type(agent))
grid_rewards = []
# 开始计时
start_time = tm.time()

episode_reward = 0
ACTIONS = []
base_revenue = []
first_revnue = []
base_vehicle = []
base_vehicle_class = []
base_order_class = []
base_city_node = []
train_rewards = []
burn_in = 100
batch_time = int(TIME)
greedy_epsilon = 0.7
for episode in range(NUM_EPISODES):
    print(f"第{episode}次训练")
    Total_order = copy.deepcopy(prim_order)
    Vehicles = copy.deepcopy(prim_vehicle)
    objval = 0
    total_objval = 0
    reward = 0
    episode_reward = 0
    invalid_time =  0
    orders_unmatched = {} # 忘记加这个了
    orders_virtual = {}
    
    if episode > 0:
        if episode > 1:
            env.time = 0
        
        if episode < burn_in:
            explore = True
        else:
            greedy_epsilon = greedy_epsilon * greedy_epsilon
            explore = False
    if_end = False
    for time in range(batch_time):
    
        group = [[], []]
        if time == batch_time -1:
            if_end = True
        # 按时间给出订单
        for order in Total_order.values():
            if order.start_time == time:
                orders_unmatched[order.id] = order
            # 加上这个代码后会导致性能降低
        for vehicle in Vehicles.values():
            if vehicle.whether_city:
                group[0].append(vehicle.id)
            else:
                group[1].append(vehicle.id)
        
        if time != 0 and episode != 0:
            
            next_vehicle_states = vectorization_vehicle(Vehicles)
            # 改了，不再是total_order
            next_order_states = vectorization_order_mask(orders_unmatched, G, num_city)
            # 这里防止梯度爆炸缩小了reward
            if last_vehicle:
                grid_reward = np.clip(grid_reward, 0, 2)
                agent.store_experience(vehicle_states, order_states, selected_log_probs, 
                       log_probs, probs, grid_reward, next_vehicle_states, next_order_states)
                agent.update_third_buffer_rnn()
            env.time = time
        if time == 0 :
            orders_virtual = orders_unmatched
           
            city_node = city_node_generator(G, orders_virtual, Vehicles, orders_unmatched)
            if episode == 1 :
                env = DispatchEnv(
                    G=G,
                    vehicles=Vehicles,
                    orders=Total_order,
                    cities=city_node,
                    capacity=CAPACITY
                )
            elif episode > 1:
                env.cities = city_node
           
            
        else:
            if episode == 0:
                city_update_without_drl(city_node , Vehicles, orders_unmatched ,time)
            else:
                city_update_without_drl(env.cities , Vehicles, orders_unmatched, time)
            
        if episode != 0:
            
            vehicle_states = vectorization_vehicle(Vehicles)
            # 这里也改了
            order_states = vectorization_order_mask(orders_unmatched, G, num_city)
            if len(group[0]) != 0:
                greedy = random.randint(0, 1)
                if greedy > greedy_epsilon:
                    greedy = True
                mask = env.get_mask(orders_unmatched)
                action, selected_log_probs, log_probs, probs = agent.take_action_third(vehicle_states, order_states, mask, explore, greedy)
                reward = env.test_step(orders_unmatched,action)
            
                ACTIONS.append(action) 

        

        if len(group[0]) != 0:
            last_vehicle = True
            if episode == 0:
                temp_Lower_Layer = Lower_Layer(G, city_node, Vehicles, orders_unmatched, name, group, time)
            else:
                temp_Lower_Layer = Lower_Layer(G, env.cities, Vehicles, orders_unmatched, name, group, time)
            try:
                temp_Lower_Layer.get_decision()
                temp_Lower_Layer.constrain_1()
                temp_Lower_Layer.constrain_2()
                temp_Lower_Layer.constrain_3()
                temp_Lower_Layer.constrain_4()
                temp_Lower_Layer.constrain_5()
                temp_Lower_Layer.model.setParam('OutputFlag', 0)
                total_penalty = cancel_penalty * order_canceled
                temp_Lower_Layer.set_objective(matrix)
            
                temp_Lower_Layer.model.optimize()

                if temp_Lower_Layer.model.status == GRB.OPTIMAL:
                    # save_results(temp_Lower_Layer,time)
                    # print("Objective value:", temp_Lower_Layer.model.objVal)
                    objval = temp_Lower_Layer.model.objVal 
                else:
                    temp_Lower_Layer.model.computeIIS()
                    temp_Lower_Layer.model.write('iis.ilp')  # 保存不可行约束
                    # print(f"{time}次，No optimal solution found.")
                    self_update(Vehicles, G)
                    objval = basic_cost(Vehicles, orders_unmatched)
                    
                
                _, var_order = temp_Lower_Layer.get_decision()
                update_var(temp_Lower_Layer, Vehicles, orders_unmatched)
                vehicle_in_city = update_vehicle(Vehicles, battery_consume, battery_add, speed, G)
                order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
            except:
                self_update(Vehicles, G)
               
                order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
                objval = - basic_cost(Vehicles, orders_unmatched)
                invalid_time += 1
            
        else :
            last_vehicle = False
            self_update(Vehicles, G)
            order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
            objval = - basic_cost(Vehicles, orders_unmatched)
            invalid_time += 1
        objval = objval - update_order(orders_unmatched, time, speed) * cancel_penalty
        if episode != 0: 
            
            grid_reward =   objval/1000
            grid_rewards.append(reward)
            episode_reward += reward + objval
        total_objval += objval

        if episode == 0:
            base_revenue.append(objval)
            base_vehicle.append(copy.deepcopy(group[0]))
            base_city_node.append(copy.deepcopy(city_node))
           
       
    end_time = tm.time()
    execution_time = end_time - start_time
    if episode != 0:
        print(f"执行时间: {execution_time} 秒,{invalid_time}次未求解，当前强化学习值为{episode_reward},利润为{total_objval}")
        save_path = f"rl_para/model_checkpoint_{episode}.pth"
        torch.save(agent, save_path)
        train_rewards.append(total_objval)
        first_invalid = invalid_time
    else:
        print(f"未加强化学习利润为{total_objval},{invalid_time}次未求解")

## 实验记录

1. 基模型：不收敛
2. greedy_epsilon二次收敛，模型永远更新参数
3. 改变了参数的传递，使得take_action_third()返回当前掩码下的概率分布，同时将这些概率传入update_third()，因此在对应使用中不再是之前的重新计算。
4. 优化模型不求解则强化学习亦不更新参数
5. update采用RNN


# 画图

In [ ]:
import plotly.express as px
import pandas as pd
px.line(pd.DataFrame({'Iteration': range(len(train_rewards)), 'Reward': train_rewards}), 
        x='Iteration', y='Reward', title='Total Reward Curve').show()

INFO:numexpr.utils:NumExpr defaulting to 16 threads.


# 新智能体

**新智能体的设计逻辑**
1. 将订单按照最终可行的城市集分为多类
2. 为每一个类设计一个智能体
3. 智能体只在有对应订单时才进行强化学习
4. 智能体的奖励来自于订单执行后的利润

In [ ]:
import csv
import numpy as np
from gurobipy import *
from CITY_GRAPH import *
from CITY_NODE import *
from ORDER import *
from VEHICLE import *
from tool_func import *
from Lower_Layer import *
import importlib
import tool_func
from update import *
import os
import time as tm
import copy
from my_env import *
import torch
import multiagent as magent
import matplotlib.pyplot as plt
import pickle

with open('sample_data.pkl', 'rb') as f:
    data = pickle.load(f)
    Vehicles = data['Vehicles']
    Total_order = data['Total_order']
    G = data['G']
# 初始化
num_vehicle = 20
num_order = 6
num_city = 8
TIME = 144  # 
CAPACITY = 7
row = [10, 1, 3, 10]
speed = 20 # 之前是20
cancel_penalty = 300
battery_consume = 10
battery_add = 300

matrix = np.tile(row, (num_vehicle, 1))
orders_unmatched = {}
name = "navie"
cancel_penalty = 300
order_canceled = 0


# 深复制最初的订单与车辆
prim_order = copy.deepcopy(Total_order)
prim_vehicle = copy.deepcopy(Vehicles)
"""这里是强化学习部分"""
# 超参数
STATE_DIM_VEHICLE = 11   # 车辆状态的特征维度
STATE_DIM_ORDER = 12     # 订单状态的特征维度
HIDDEN_DIM = 128         # 隐藏层维度
ACTION_DIM = num_city          # 动作空间维度
ACTOR_LR = 1e-2          # Actor 学习率
CRITIC_LR = 1e-2         # Critic 学习率
GAMMA = 0.99             # 折扣因子
NUM_EPISODES = 10     # 总训练轮数
# 这里也改了
STATE_DIM = 2 *HIDDEN_DIM       
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
action_types = order_same_action(Total_order, num_city, G)
AGENT = {}
for action_type, same_orders in action_types.items():
    agent = magent.MultiAgentAC(
        device = DEVICE,
        VEHICLE_STATE_DIM = STATE_DIM_VEHICLE,
        ORDER_STATE_DIM = STATE_DIM_ORDER, 
        NUM_CITIES = len(action_type), 
        HIDDEN_DIM = HIDDEN_DIM, 
        STATE_DIM = STATE_DIM
    )
    agent.action_key = action_type
    AGENT[action_type] = agent

grid_rewards = []
# 开始计时
start_time = tm.time()

episode_reward = 0
ACTIONS = []
base_revenue = []
first_revnue = []
base_vehicle = []
base_vehicle_class = []
base_order_class = []
base_city_node = []
train_rewards = []
burn_in = 100
batch_time = int(TIME)
greedy_epsilon = 0.7
for episode in range(NUM_EPISODES):
    print(f"第{episode}次训练")
    Total_order = copy.deepcopy(prim_order)
    Vehicles = copy.deepcopy(prim_vehicle)
    objval = 0
    total_objval = 0
    reward = 0
    episode_reward = 0
    invalid_time =  0
    orders_unmatched = {} # 忘记加这个了
    orders_virtual = {}
    
    if episode > 0:
        if episode > 1:
            env.time = 0
        
        if episode < burn_in:
            explore = True
        else:
            greedy_epsilon = greedy_epsilon * greedy_epsilon
            explore = False
    if_end = False
    for time in range(batch_time):
    
        group = [[], []]
        if time == batch_time -1:
            if_end = True
        # 按时间给出订单
        for order in Total_order.values():
            if order.start_time == time:
                orders_unmatched[order.id] = order
            # 加上这个代码后会导致性能降低
        for vehicle in Vehicles.values():
            if vehicle.whether_city:
                group[0].append(vehicle.id)
            else:
                group[1].append(vehicle.id)
        # 判断订单当前是否激活，并且赋值当前激活的订单给current_order
        for agent in AGENT.values():
            active_test(agent.action_key, agent, orders_unmatched)
            get_multi_reward(agent)

        if time != 0 and episode != 0:
            
            next_vehicle_states = vectorization_vehicle(Vehicles)
            
            # 这里防止梯度爆炸缩小了reward
            if last_vehicle:
                for agent in AGENT.values():
                    if agent.active:
                        next_order_states = vectorization_order(agent.current_order)
                        if agent.last_order:
                            order_states = vectorization_order(agent.last_order)
                        else:
                            continue
                            # 过去time=0确保了order_states存在，现在需要第一次训练后才有
                        # 改一下grid_reward
                        # print(agent.action_key,time, agent.reward)
                        agent.update(agent.v_states, order_states, agent.action, 
                                    agent.reward + objval, next_vehicle_states, next_order_states , if_end)
                    
            env.time = time

        

        if time == 0 :
            orders_virtual = orders_unmatched
           
            city_node = city_node_generator(G, orders_virtual, Vehicles, orders_unmatched)
            if episode == 1 :
                env = DispatchEnv(
                    G=G,
                    vehicles=Vehicles,
                    orders=Total_order,
                    cities=city_node,
                    capacity=CAPACITY
                )
            elif episode > 1:
                env.cities = city_node
           
            
        else:
            if episode == 0:
                city_update_without_drl(city_node , Vehicles, orders_unmatched ,time)
            else:
                city_update_without_drl(env.cities , Vehicles, orders_unmatched, time)
            
        if episode != 0:
            if len(group[0]) != 0:
                greedy = random.randint(0, 1)
                # if greedy > greedy_epsilon:
                greedy = False
                vehicle_states = vectorization_vehicle(Vehicles)
                # 这里也改了
                
                for agent in AGENT.values():
                    if agent.active:
                        agent.v_states = vehicle_states
                        order_states = vectorization_order(agent.current_order)
                        agent.action= agent.take_action_skyrim(agent.v_states, order_states, explore, greedy)
                        # ACTIONS.append(agent.action) 
        if len(group[0]) != 0:
            last_vehicle = True
            if episode == 0:
                temp_Lower_Layer = Lower_Layer(G, city_node, Vehicles, orders_unmatched, name, group, time)
            else:
                temp_Lower_Layer = Lower_Layer(G, env.cities, Vehicles, orders_unmatched, name, group, time)
            try:
                temp_Lower_Layer.get_decision()
                temp_Lower_Layer.constrain_1()
                temp_Lower_Layer.constrain_2()
                temp_Lower_Layer.constrain_3()
                temp_Lower_Layer.constrain_4()
                temp_Lower_Layer.constrain_5()
                temp_Lower_Layer.model.setParam('OutputFlag', 0)
                total_penalty = cancel_penalty * order_canceled
                temp_Lower_Layer.set_objective(matrix)
            
                temp_Lower_Layer.model.optimize()

                if temp_Lower_Layer.model.status == GRB.OPTIMAL:
                    # save_results(temp_Lower_Layer,time)
                    # print("Objective value:", temp_Lower_Layer.model.objVal)
                    objval = temp_Lower_Layer.model.objVal 
                else:
                    temp_Lower_Layer.model.computeIIS()
                    temp_Lower_Layer.model.write('iis.ilp')  # 保存不可行约束
                    # print(f"{time}次，No optimal solution found.")
                    self_update(Vehicles, G)
                    objval = basic_cost(Vehicles, orders_unmatched)
                    
                
                _, var_order = temp_Lower_Layer.get_decision()
                update_var(temp_Lower_Layer, Vehicles, orders_unmatched)
                vehicle_in_city = update_vehicle(Vehicles, battery_consume, battery_add, speed, G)
                order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
            except:
                self_update(Vehicles, G)
               
                order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
                objval = - basic_cost(Vehicles, orders_unmatched)
                invalid_time += 1
            
        else:
            last_vehicle = False
            self_update(Vehicles, G)
            order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
            objval = - basic_cost(Vehicles, orders_unmatched)
            invalid_time += 1
        objval = objval - update_order(orders_unmatched, time, speed) * cancel_penalty
        if episode != 0: 
            
            grid_reward =   objval
            grid_rewards.append(reward)
            episode_reward += reward + objval
        total_objval += objval

    end_time = tm.time()
    execution_time = end_time - start_time
    if episode != 0:
        print(f"执行时间: {execution_time} 秒,{invalid_time}次未求解，当前强化学习值为{episode_reward},利润为{total_objval}")
        save_path = f"rl_para/model_checkpoint_{episode}.pth"
        torch.save(agent, save_path)
        train_rewards.append(total_objval)
        first_invalid = invalid_time        
    else:
        print(f"未加强化学习利润为{total_objval},{invalid_time}次未求解")

c:\ProgramData\Anaconda3\envs\dirty_test\lib\site-packages\torch\nn\init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


第0次训练


AttributeError: 'MultiAgentAC' object has no attribute 'current_order'

# 赢麻了

In [ ]:
import csv
import numpy as np
from gurobipy import *
from CITY_GRAPH import *
from CITY_NODE import *
from ORDER import *
from VEHICLE import *
from tool_func import *
from Lower_Layer import *
import importlib

from update import *
import os
import time as tm
import copy
from my_env import *
import torch
import aloha as magent
import matplotlib.pyplot as plt
import pickle

with open('sample_data.pkl', 'rb') as f:
    data = pickle.load(f)
    Vehicles = data['Vehicles']
    Total_order = data['Total_order']
    G = data['G']
# 初始化
num_vehicle = 20
num_order = 6
num_city = 8
TIME = 144  # 
CAPACITY = 7
row = [10, 1, 3, 10]
speed = 20 # 之前是20
cancel_penalty = 300
battery_consume = 10
battery_add = 300

matrix = np.tile(row, (num_vehicle, 1))
orders_unmatched = {}
name = "navie"
cancel_penalty = 300
order_canceled = 0


# 深复制最初的订单与车辆
prim_order = copy.deepcopy(Total_order)
prim_vehicle = copy.deepcopy(Vehicles)
"""这里是强化学习部分"""
# 超参数
STATE_DIM_VEHICLE = 11   # 车辆状态的特征维度
STATE_DIM_ORDER = 12     # 订单状态的特征维度
HIDDEN_DIM = 128         # 隐藏层维度
ACTION_DIM = num_city          # 动作空间维度
ACTOR_LR = 1e-2          # Actor 学习率
CRITIC_LR = 1e-2         # Critic 学习率
GAMMA = 0.99             # 折扣因子
NUM_EPISODES = 300     # 总训练轮数
# 这里也改了
STATE_DIM = 2 *HIDDEN_DIM       
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
agent = magent.MultiAgentAC(
    device = DEVICE,
    VEHICLE_STATE_DIM = STATE_DIM_VEHICLE,
    ORDER_STATE_DIM = STATE_DIM_ORDER, 
    NUM_CITIES = num_city, 
    HIDDEN_DIM = HIDDEN_DIM, 
    STATE_DIM = num_city * 2
)
print(type(agent))
grid_rewards = []
# 开始计时
start_time = tm.time()

episode_reward = 0
ACTIONS = []
base_revenue = []
first_revnue = []
base_vehicle = []
base_vehicle_class = []
base_order_class = []
base_city_node = []
train_rewards = []
burn_in = 100
batch_time = int(TIME)
greedy_epsilon = 0.7
for episode in range(NUM_EPISODES):
    print(f"第{episode}次训练")
    Total_order = copy.deepcopy(prim_order)
    Vehicles = copy.deepcopy(prim_vehicle)
    objval = 0
    total_objval = 0
    reward = 0
    episode_reward = 0
    invalid_time =  0
    orders_unmatched = {} # 忘记加这个了
    orders_virtual = {}
    
    if episode > 0:
        if episode > 1:
            env.time = 0
        
        if episode < burn_in:
            explore = True
        else:
            greedy_epsilon = greedy_epsilon * greedy_epsilon
            explore = False
    if_end = False
    for time in range(batch_time):
    
        group = [[], []]
        if time == batch_time -1:
            if_end = True
        # 按时间给出订单
        for order in Total_order.values():
            if order.start_time == time:
                orders_unmatched[order.id] = order
            # 加上这个代码后会导致性能降低
        for vehicle in Vehicles.values():
            if vehicle.whether_city:
                group[0].append(vehicle.id)
            else:
                group[1].append(vehicle.id)
        
        if time != 0 and episode != 0:
            
            next_vehicle_states = seat_count(CAPACITY,city_node)
            # 改了，不再是total_order
            next_order_states = vectorization_order_mask(orders_unmatched, G, num_city)
            # 这里防止梯度爆炸缩小了reward
            agent.update_third(vehicle_states, order_states, action, selected_log_probs, log_probs, probs,
                        grid_reward, next_vehicle_states, next_order_states , if_end)
            env.time = time
        if time == 0 :
            orders_virtual = orders_unmatched
           
            city_node = city_node_generator(G, orders_virtual, Vehicles, orders_unmatched)
            if episode == 1 :
                env = DispatchEnv(
                    G=G,
                    vehicles=Vehicles,
                    orders=Total_order,
                    cities=city_node,
                    capacity=CAPACITY
                )
            elif episode > 1:
                env.cities = city_node
           
            
        else:
            if episode == 0:
                city_update_without_drl(city_node , Vehicles, orders_unmatched ,time)
            else:
                city_update_without_drl(env.cities , Vehicles, orders_unmatched, time)
            
        if episode != 0:
            
            vehicle_states = seat_count(CAPACITY,city_node)
            # 这里也改了
            order_states = vectorization_order_mask(orders_unmatched, G, num_city)
            greedy = random.randint(0, 1)
            if greedy > greedy_epsilon:
                greedy = True
            mask = env.get_mask(orders_unmatched)
            # 注意这里有多种take_action形式
            action, selected_log_probs, log_probs, probs = agent.take_action_vehicle(vehicle_states, order_states, mask, explore, greedy)
            reward = env.test_step(orders_unmatched,action)
        
            ACTIONS.append(action) 

        

        if len(group[0]) != 0:
            last_vehicle = True
            if episode == 0:
                temp_Lower_Layer = Lower_Layer(G, city_node, Vehicles, orders_unmatched, name, group, time)
            else:
                temp_Lower_Layer = Lower_Layer(G, env.cities, Vehicles, orders_unmatched, name, group, time)
            try:
                temp_Lower_Layer.get_decision()
                temp_Lower_Layer.constrain_1()
                temp_Lower_Layer.constrain_2()
                temp_Lower_Layer.constrain_3()
                temp_Lower_Layer.constrain_4()
                temp_Lower_Layer.constrain_5()
                temp_Lower_Layer.model.setParam('OutputFlag', 0)
                total_penalty = cancel_penalty * order_canceled
                temp_Lower_Layer.set_objective(matrix)
            
                temp_Lower_Layer.model.optimize()

                if temp_Lower_Layer.model.status == GRB.OPTIMAL:
                    # save_results(temp_Lower_Layer,time)
                    # print("Objective value:", temp_Lower_Layer.model.objVal)
                    objval = temp_Lower_Layer.model.objVal 
                else:
                    temp_Lower_Layer.model.computeIIS()
                    temp_Lower_Layer.model.write('iis.ilp')  # 保存不可行约束
                    # print(f"{time}次，No optimal solution found.")
                    self_update(Vehicles, G)
                    objval = basic_cost(Vehicles, orders_unmatched)
                    
                
                _, var_order = temp_Lower_Layer.get_decision()
                update_var(temp_Lower_Layer, Vehicles, orders_unmatched)
                vehicle_in_city = update_vehicle(Vehicles, battery_consume, battery_add, speed, G)
                order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
            except:
                self_update(Vehicles, G)
               
                order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
                objval = - basic_cost(Vehicles, orders_unmatched)
                invalid_time += 1
            
        else :
            last_vehicle = False
            self_update(Vehicles, G)
            order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
            objval = - basic_cost(Vehicles, orders_unmatched)
            invalid_time += 1
        objval = objval - update_order(orders_unmatched, time, speed) * cancel_penalty
        if episode != 0: 
            
            # grid_reward =   (objval-base_revenue[time])/1000
            grid_reward =  objval /1000
            grid_rewards.append(reward)
            episode_reward += reward + objval
        total_objval += objval

        if episode == 0:
            base_revenue.append(objval)
            base_vehicle.append(copy.deepcopy(group[0]))
            base_city_node.append(copy.deepcopy(city_node))
           
       
    end_time = tm.time()
    execution_time = end_time - start_time
    if episode != 0:
        print(f"执行时间: {execution_time} 秒,{invalid_time}次未求解，当前强化学习值为{episode_reward},利润为{total_objval}")
        save_path = f"rl_para/model_checkpoint_{episode}.pth"
        torch.save(agent, save_path)
        train_rewards.append(total_objval)
        first_invalid = invalid_time
    else:
        print(f"未加强化学习利润为{total_objval},{invalid_time}次未求解")

<class 'aloha.MultiAgentAC'>
第0次训练
Set parameter LicenseID to value 2584673


INFO:gurobipy:Set parameter LicenseID to value 2584673


未加强化学习利润为569776.0,101次未求解
第1次训练
tensor([[0.0000, 0.3108, 0.0000, 0.0000, 0.0000, 0.3626, 0.0000, 0.3266],
        [0.5317, 0.0000, 0.0000, 0.0000, 0.0000, 0.4683, 0.0000, 0.0000],
        [0.0000, 0.3170, 0.0000, 0.3124, 0.0000, 0.3706, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4073, 0.0000, 0.3086, 0.0000, 0.2841, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.3069, 0.0000, 0.3658, 0.0000, 0.3273]],
       grad_fn=<NanToNumBackward0>) tensor([[ 0.0000, -1.1686,  0.0000,  0.0000,  0.0000, -1.0146,  0.0000, -1.1189],
        [-0.6317,  0.0000,  0.0000,  0.0000,  0.0000, -0.7586,  0.0000,  0.0000],
        [ 0.0000, -1.1489,  0.0000, -1.1635,  0.0000, -0.9926,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.8982,  0.0000, -1.1757,  0.0000, -1.2585,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000, -1.1814,  0.0000, -1.0057,  0.0000, -1.11

In [ ]:
import plotly.express as px
import pandas as pd
px.line(pd.DataFrame({'Iteration': range(len(train_rewards)), 'Reward': train_rewards}), 
        x='Iteration', y='Reward', title='Total Reward Curve').show()

# aloha buffer

In [ ]:
import csv
import numpy as np
from gurobipy import *
from CITY_GRAPH import *
from CITY_NODE import *
from ORDER import *
from VEHICLE import *
from tool_func import *
from Lower_Layer import *
import importlib

from update import *
import os
import time as tm
import copy
from my_env import *
import torch
import aloha_buffer as magent
import matplotlib.pyplot as plt
import pickle

with open('sample_data.pkl', 'rb') as f:
    data = pickle.load(f)
    Vehicles = data['Vehicles']
    Total_order = data['Total_order']
    G = data['G']
# 初始化
num_vehicle = 20
num_order = 6
num_city = 8
TIME = 144  # 
CAPACITY = 7
row = [10, 1, 3, 10]
speed = 20 # 之前是20
cancel_penalty = 300
battery_consume = 10
battery_add = 300

matrix = np.tile(row, (num_vehicle, 1))
orders_unmatched = {}
name = "navie"
cancel_penalty = 300
order_canceled = 0


# 深复制最初的订单与车辆
prim_order = copy.deepcopy(Total_order)
prim_vehicle = copy.deepcopy(Vehicles)
"""这里是强化学习部分"""
# 超参数
STATE_DIM_VEHICLE = 11   # 车辆状态的特征维度
STATE_DIM_ORDER = 12     # 订单状态的特征维度
HIDDEN_DIM = 32         # 隐藏层维度
ACTION_DIM = num_city          # 动作空间维度
ACTOR_LR = 1e-2          # Actor 学习率
CRITIC_LR = 1e-2         # Critic 学习率
GAMMA = 0.99             # 折扣因子
NUM_EPISODES = 300     # 总训练轮数
# 这里也改了
STATE_DIM = 2 *HIDDEN_DIM       
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
agent = magent.MultiAgentAC(
    device = DEVICE,
    VEHICLE_STATE_DIM = STATE_DIM_VEHICLE,
    ORDER_STATE_DIM = STATE_DIM_ORDER, 
    NUM_CITIES = num_city, 
    HIDDEN_DIM = HIDDEN_DIM, 
    STATE_DIM = num_city*2 
)
print(type(agent))
grid_rewards = []
# 开始计时
start_time = tm.time()

episode_reward = 0
ACTIONS = []
base_revenue = []
first_revnue = []
base_vehicle = []
base_vehicle_class = []
base_order_class = []
base_city_node = []
train_rewards = []
burn_in = 100
batch_time = int(TIME)
greedy_epsilon = 0.7
for episode in range(NUM_EPISODES):
    print(f"第{episode}次训练")
    Total_order = copy.deepcopy(prim_order)
    Vehicles = copy.deepcopy(prim_vehicle)
    objval = 0
    total_objval = 0
    reward = 0
    episode_reward = 0
    invalid_time =  0
    orders_unmatched = {} # 忘记加这个了
    orders_virtual = {}
    
    if episode > 0:
        if episode > 1:
            env.time = 0
        
        if episode < burn_in:
            explore = True
        else:
            greedy_epsilon = greedy_epsilon * greedy_epsilon
            explore = False
    if_end = False
    for time in range(batch_time):
    
        group = [[], []]
        if time == batch_time -1:
            if_end = True
        # 按时间给出订单
        for order in Total_order.values():
            if order.start_time == time:
                orders_unmatched[order.id] = order
            # 加上这个代码后会导致性能降低
        for vehicle in Vehicles.values():
            if vehicle.whether_city:
                group[0].append(vehicle.id)
            else:
                group[1].append(vehicle.id)
        
        if time != 0 and episode != 0:
            
            next_vehicle_states = seat_count(CAPACITY,city_node)
            # 改了，不再是total_order
            next_order_states = vectorization_order_mask(orders_unmatched, G, num_city)
            # 这里防止梯度爆炸缩小了reward
            agent.update(env.time)
            agent.store_experience(vehicle_states, order_states, selected_log_probs, action,
                       log_probs, probs, grid_reward, next_vehicle_states, next_order_states, if_end)
            env.time = time 
        if time == 0 :
            orders_virtual = orders_unmatched
           
            city_node = city_node_generator(G, orders_virtual, Vehicles, orders_unmatched)
            if episode == 1 :
                env = DispatchEnv(
                    G=G,
                    vehicles=Vehicles,
                    orders=Total_order,
                    cities=city_node,
                    capacity=CAPACITY
                )
            elif episode > 1:
                env.cities = city_node
           
            
        else:
            if episode == 0:
                city_update_without_drl(city_node , Vehicles, orders_unmatched ,time)
            else:
                city_update_without_drl(env.cities , Vehicles, orders_unmatched, time)
            
        if episode != 0:
            
            vehicle_states = seat_count(CAPACITY,city_node)
            # 这里也改了
            order_states = vectorization_order_mask(orders_unmatched, G, num_city)
            greedy = random.randint(0, 1)
            if greedy > greedy_epsilon:
                greedy = True
            mask = env.get_mask(orders_unmatched)
            # 注意这里有多种take_action形式
            action, selected_log_probs, log_probs, probs = agent.take_action_vehicle(vehicle_states, order_states, mask, explore, greedy)
            reward = env.test_step(orders_unmatched,action)
        
            ACTIONS.append(action) 

        

        if len(group[0]) != 0:
            last_vehicle = True
            if episode == 0:
                temp_Lower_Layer = Lower_Layer(G, city_node, Vehicles, orders_unmatched, name, group, time)
            else:
                temp_Lower_Layer = Lower_Layer(G, env.cities, Vehicles, orders_unmatched, name, group, time)
            try:
                temp_Lower_Layer.get_decision()
                temp_Lower_Layer.constrain_1()
                temp_Lower_Layer.constrain_2()
                temp_Lower_Layer.constrain_3()
                temp_Lower_Layer.constrain_4()
                temp_Lower_Layer.constrain_5()
                temp_Lower_Layer.model.setParam('OutputFlag', 0)
                total_penalty = cancel_penalty * order_canceled
                temp_Lower_Layer.set_objective(matrix)
            
                temp_Lower_Layer.model.optimize()

                if temp_Lower_Layer.model.status == GRB.OPTIMAL:
                    # save_results(temp_Lower_Layer,time)
                    # print("Objective value:", temp_Lower_Layer.model.objVal)
                    objval = temp_Lower_Layer.model.objVal 
                else:
                    temp_Lower_Layer.model.computeIIS()
                    temp_Lower_Layer.model.write('iis.ilp')  # 保存不可行约束
                    # print(f"{time}次，No optimal solution found.")
                    self_update(Vehicles, G)
                    objval = basic_cost(Vehicles, orders_unmatched)
                    
                
                _, var_order = temp_Lower_Layer.get_decision()
                update_var(temp_Lower_Layer, Vehicles, orders_unmatched)
                vehicle_in_city = update_vehicle(Vehicles, battery_consume, battery_add, speed, G)
                order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
            except:
                self_update(Vehicles, G)
               
                order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
                objval = - basic_cost(Vehicles, orders_unmatched)
                invalid_time += 1
            
        else :
            last_vehicle = False
            self_update(Vehicles, G)
            order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
            objval = - basic_cost(Vehicles, orders_unmatched)
            invalid_time += 1
        objval = objval - update_order(orders_unmatched, time, speed) * cancel_penalty
        if episode != 0: 
            
            # grid_reward =   (objval-base_revenue[time])/1000
            grid_reward =  objval /1000
            grid_rewards.append(reward)
            episode_reward += reward + objval
        total_objval += objval

        if episode == 0:
            base_revenue.append(objval)
            base_vehicle.append(copy.deepcopy(group[0]))
            base_city_node.append(copy.deepcopy(city_node))
           
       
    end_time = tm.time()
    execution_time = end_time - start_time
    if episode != 0:
        print(f"执行时间: {execution_time} 秒,{invalid_time}次未求解，当前强化学习值为{episode_reward},利润为{total_objval}")
        save_path = f"rl_para/model_checkpoint_{episode}.pth"
        torch.save(agent, save_path)
        train_rewards.append(total_objval)
        first_invalid = invalid_time
    else:
        print(f"未加强化学习利润为{total_objval},{invalid_time}次未求解")

<class 'aloha_buffer.MultiAgentAC'>
第0次训练
Set parameter LicenseID to value 2584673


INFO:gurobipy:Set parameter LicenseID to value 2584673


未加强化学习利润为569776.0,101次未求解
第1次训练


d:\mycodelife\workshop\DRL_CO\aloha_buffer.py:157: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:204.)
  o_states = torch.tensor([exp.order_states for exp in batch], dtype=torch.float).to(self.device)
d:\mycodelife\workshop\DRL_CO\aloha_buffer.py:184: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  critic_loss = F.mse_loss(current_v, td_target.detach())


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

# aloha buffer 2

In [3]:
import csv
import numpy as np
from gurobipy import *
from CITY_GRAPH import *
from CITY_NODE import *
from ORDER import *
from VEHICLE import *
from tool_func import *
from Lower_Layer import *
import importlib
import tool_func
from update import *
import os
import time as tm
import copy
from my_env import *
import torch
import multiagent as magent
import matplotlib.pyplot as plt
import pickle
# 假设有如下初始化函数

"""这里是非强化学习部分"""
# 初始化
num_vehicle = 10
num_order = 2
num_city = 6
TIME = 144  # 
CAPACITY = 7
row = [10, 1, 3, 10]
Vehicles = {}
speed = 20 # 之前是20
cancel_penalty = 300
battery_consume = 10
battery_add = 300

matrix = np.tile(row, (num_vehicle, 1))
Vehicles = vehicle_generator(num_vehicle, num_city)
orders_unmatched = {}
G = CityGraph(num_city, 0.3, (10, 30))
name = "navie"
cancel_penalty = 300
order_canceled = 0
Total_order = {}

# 生成固定样本
for time in range(TIME):
    Orders = order_generator(num_order, time, num_city-1, CAPACITY, G, speed)
    for order in Orders.values():
        Total_order[order.id] = order

# 保存样本到本地
with open('sample_data.pkl', 'wb') as f:
    pickle.dump({
        'Vehicles': Vehicles,
        'Total_order': Total_order,
        'G':G
    }, f)

In [6]:
import csv
import numpy as np
from gurobipy import *
from CITY_GRAPH import *
from CITY_NODE import *
from ORDER import *
from VEHICLE import *
from tool_func import *
from Lower_Layer import *
import importlib

from update import *
import os
import time as tm
import copy
from my_env import *
import torch
import aloha_buffer_2 as magent
import matplotlib.pyplot as plt
import pickle

with open('sample_data.pkl', 'rb') as f:
    data = pickle.load(f)
    Vehicles = data['Vehicles']
    Total_order = data['Total_order']
    G = data['G']
# 初始化
num_vehicle = 10
num_order = 2
num_city = 6
TIME = 144  # 
CAPACITY = 7
row = [10, 1, 3, 10]
speed = 20 # 之前是20
cancel_penalty = 300
battery_consume = 10
battery_add = 300

matrix = np.tile(row, (num_vehicle, 1))
orders_unmatched = {}
name = "navie"
cancel_penalty = 300
order_canceled = 0


# 深复制最初的订单与车辆
prim_order = copy.deepcopy(Total_order)
prim_vehicle = copy.deepcopy(Vehicles)
"""这里是强化学习部分"""
# 超参数
STATE_DIM_VEHICLE = 11   # 车辆状态的特征维度
STATE_DIM_ORDER = 12     # 订单状态的特征维度
HIDDEN_DIM = 32         # 隐藏层维度
ACTION_DIM = num_city          # 动作空间维度
ACTOR_LR = 1e-2          # Actor 学习率
CRITIC_LR = 1e-2         # Critic 学习率
GAMMA = 0.99             # 折扣因子
NUM_EPISODES = 300     # 总训练轮数
# 这里也改了
STATE_DIM = 2 *HIDDEN_DIM
batch_size = 16       
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
agent = magent.MultiAgentAC(
    device = DEVICE,
    VEHICLE_STATE_DIM = STATE_DIM_VEHICLE,
    ORDER_STATE_DIM = STATE_DIM_ORDER, 
    NUM_CITIES = num_city, 
    HIDDEN_DIM = HIDDEN_DIM, 
    STATE_DIM = num_city*2,
    batch_size = batch_size
)
print(type(agent))
grid_rewards = []
# 开始计时
start_time = tm.time()

episode_reward = 0
ACTIONS = []
base_revenue = []
first_revnue = []
base_vehicle = []
base_vehicle_class = []
base_order_class = []
base_city_node = []
train_rewards = []
burn_in = 100
batch_time = int(TIME)
greedy_epsilon = 0.7
for episode in range(NUM_EPISODES):
    print(f"第{episode}次训练")
    Total_order = copy.deepcopy(prim_order)
    Vehicles = copy.deepcopy(prim_vehicle)
    objval = 0
    total_objval = 0
    reward = 0
    episode_reward = 0
    invalid_time =  0
    orders_unmatched = {} # 忘记加这个了
    orders_virtual = {}
    
    if episode > 0:
        if episode > 1:
            env.time = 0
        
        if episode < burn_in:
            explore = True
        else:
            greedy_epsilon = greedy_epsilon * greedy_epsilon
            explore = False
    if_end = False
    for time in range(batch_time):
    
        group = [[], []]
        if time == batch_time -1:
            if_end = True
        # 按时间给出订单
        for order in Total_order.values():
            if order.start_time == time:
                orders_unmatched[order.id] = order
            # 加上这个代码后会导致性能降低
        for vehicle in Vehicles.values():
            if vehicle.whether_city:
                group[0].append(vehicle.id)
            else:
                group[1].append(vehicle.id)
        
        if time != 0 and episode != 0:
            agent.store_experience(vehicle_states, order_states, grid_reward, probs, log_probs, selected_log_probs)
            # 这里防止梯度爆炸缩小了reward
            agent.update(env.time)
            
            env.time = time 
        if time == 0 :
            orders_virtual = orders_unmatched
           
            city_node = city_node_generator(G, orders_virtual, Vehicles, orders_unmatched)
            if episode == 1 :
                env = DispatchEnv(
                    G=G,
                    vehicles=Vehicles,
                    orders=Total_order,
                    cities=city_node,
                    capacity=CAPACITY
                )
            if episode > 1:
                for i in range(int(batch_size/2)):
                    agent.update(i+TIME-batch_size)
                agent.buffer.clear()
            elif episode > 1:
                env.cities = city_node
           
            
        else:
            if episode == 0:
                city_update_without_drl(city_node , Vehicles, orders_unmatched ,time)
            else:
                city_update_without_drl(env.cities , Vehicles, orders_unmatched, time)
            
        if episode != 0:
            
            vehicle_states = seat_count(CAPACITY,city_node)
            # 这里也改了
            order_states = vectorization_order_mask(orders_unmatched, G, num_city)
            greedy = random.randint(0, 1)
            if greedy > greedy_epsilon:
                greedy = True
            mask = env.get_mask(orders_unmatched)
            # 注意这里有多种take_action形式
            action, selected_log_probs, log_probs, probs = agent.take_action_vehicle(vehicle_states, order_states, mask, explore, greedy)
            
            reward = env.test_step(orders_unmatched,action)
        
            ACTIONS.append(action) 

        

        if len(group[0]) != 0:
            last_vehicle = True
            if episode == 0:
                temp_Lower_Layer = Lower_Layer(G, city_node, Vehicles, orders_unmatched, name, group, time)
            else:
                temp_Lower_Layer = Lower_Layer(G, env.cities, Vehicles, orders_unmatched, name, group, time)
            try:
                temp_Lower_Layer.get_decision()
                temp_Lower_Layer.constrain_1()
                temp_Lower_Layer.constrain_2()
                temp_Lower_Layer.constrain_3()
                temp_Lower_Layer.constrain_4()
                temp_Lower_Layer.constrain_5()
                temp_Lower_Layer.model.setParam('OutputFlag', 0)
                total_penalty = cancel_penalty * order_canceled
                temp_Lower_Layer.set_objective(matrix)
            
                temp_Lower_Layer.model.optimize()

                if temp_Lower_Layer.model.status == GRB.OPTIMAL:
                    # save_results(temp_Lower_Layer,time)
                    # print("Objective value:", temp_Lower_Layer.model.objVal)
                    objval = temp_Lower_Layer.model.objVal 
                else:
                    temp_Lower_Layer.model.computeIIS()
                    temp_Lower_Layer.model.write('iis.ilp')  # 保存不可行约束
                    # print(f"{time}次，No optimal solution found.")
                    self_update(Vehicles, G)
                    objval = basic_cost(Vehicles, orders_unmatched)
                    
                
                _, var_order = temp_Lower_Layer.get_decision()
                update_var(temp_Lower_Layer, Vehicles, orders_unmatched)
                vehicle_in_city = update_vehicle(Vehicles, battery_consume, battery_add, speed, G)
                order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
            except:
                self_update(Vehicles, G)
               
                order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
                objval = - basic_cost(Vehicles, orders_unmatched)
                invalid_time += 1
            
        else :
            last_vehicle = False
            self_update(Vehicles, G)
            order_canceled = order_canceled + update_order(orders_unmatched, time, speed)
            objval = - basic_cost(Vehicles, orders_unmatched)
            invalid_time += 1
        objval = objval - update_order(orders_unmatched, time, speed) * cancel_penalty
        if episode != 0: 
            
            # grid_reward =   (objval-base_revenue[time])/1000
            grid_reward =  objval /1000
            grid_rewards.append(reward)
            episode_reward += reward + objval
        total_objval += objval

        if episode == 0:
            base_revenue.append(objval)
            base_vehicle.append(copy.deepcopy(group[0]))
            base_city_node.append(copy.deepcopy(city_node))
           
       
    end_time = tm.time()
    execution_time = end_time - start_time
    if episode != 0:
        print(f"执行时间: {execution_time} 秒,{invalid_time}次未求解，当前强化学习值为{episode_reward},利润为{total_objval}")
        save_path = f"rl_para/model_checkpoint_{episode}.pth"
        torch.save(agent, save_path)
        train_rewards.append(total_objval)
        first_invalid = invalid_time
    else:
        print(f"未加强化学习利润为{total_objval},{invalid_time}次未求解")


<class 'aloha_buffer_2.MultiAgentAC'>
第0次训练
未加强化学习利润为685942.0,8次未求解
第1次训练
执行时间: 1.0903592109680176 秒,73次未求解，当前强化学习值为319904.0,利润为319904.0
第2次训练
执行时间: 1.8839962482452393 秒,0次未求解，当前强化学习值为732050.0,利润为732050.0
第3次训练
执行时间: 2.300845146179199 秒,125次未求解，当前强化学习值为19972.0,利润为19972.0
第4次训练
执行时间: 2.9358794689178467 秒,56次未求解，当前强化学习值为420371.0,利润为420371.0
第5次训练
执行时间: 3.7301454544067383 秒,0次未求解，当前强化学习值为731892.0,利润为731892.0
第6次训练
执行时间: 4.160373687744141 秒,128次未求解，当前强化学习值为-3613.0,利润为-3613.0
第7次训练
执行时间: 4.802362680435181 秒,56次未求解，当前强化学习值为420480.0,利润为420480.0
第8次训练
执行时间: 5.585039377212524 秒,0次未求解，当前强化学习值为731795.0,利润为731795.0
第9次训练
执行时间: 6.040329933166504 秒,128次未求解，当前强化学习值为-1638.0,利润为-1638.0
第10次训练
执行时间: 6.667232275009155 秒,63次未求解，当前强化学习值为379023.0,利润为379023.0
第11次训练
执行时间: 7.114333391189575 秒,127次未求解，当前强化学习值为-3729.0,利润为-3729.0
第12次训练
执行时间: 7.650373458862305 秒,94次未求解，当前强化学习值为202700.0,利润为202700.0
第13次训练
执行时间: 8.09038496017456 秒,127次未求解，当前强化学习值为-1736.0,利润为-1736.0
第14次训练
执行时间: 8.690470457077026 秒,73次未求解，当前强化学习值为3

In [7]:
import plotly.express as px
import pandas as pd
px.line(pd.DataFrame({'Iteration': range(len(train_rewards)), 'Reward': train_rewards}), 
        x='Iteration', y='Reward', title='Total Reward Curve').show()